In [1]:
cd "D:\Jupyter\Cough Classification\few-shot-ho-master"

D:\Jupyter\Cough Classification\few-shot-ho-master


In [2]:
import torch
import torchaudio
import torch.nn as nn
import os
from os import listdir
import torch.nn.functional as F
from protonets.models.encoder.baseUtil import Flatten, get_padding
from collections import OrderedDict
import numpy as np

In [3]:
import os
def cough_path(file):
    '''
    get file_path and label of this file
    '''
    label = file.split("_")[0]
#     print(label)
    file_path =os.path.join(os.path.join(path, label), file)
    return file_path, label

def build_mfcc_extractor():
    '''
    extract mfcc
    '''
    frame_len = 128/ 1000
    stride = (128/2)/ 1000
    sample_rate = 16000
    mfcc = torchaudio.transforms.MFCC(sample_rate = 16000,
                                    n_mfcc=40,
                                    melkwargs={
                                        'hop_length' : int(stride*sample_rate),
                                        'n_fft' : int(frame_len*sample_rate)})
    return mfcc
    
def load_audio(filepath):
    '''
    read audio file
    '''
    desired_samples = int(16000*1000/1000)
    sound, _ = torchaudio.load(filepath=filepath,
                                     num_frames=desired_samples)
#         d[out_field] = sound
    return sound

def getData(file):
    '''step-by-step: extract feature and get label of audio file'''
    file_path, label = cough_path(file)
    sound = load_audio(file_path)
#     print("file: ", sound)
    mfcc = build_mfcc_extractor()
#     x = mfcc(sound).reshape(1,51,40)
    features = mfcc(sound)[0]
    features = features.T # f x t -> t x f
    x = torch.unsqueeze(features,0)
#     print(x.shape)
    return x, label

def euclidean_dist(x, y):
    '''calculate Euclidean distance'''
    # x: N x D
    # y: M x D
#     print(x.shape) #1, 3, 48
    n = x.size(0)
    m = y.size(0)
    d = x.size(1)
#     print(n, m, d)
#     print("y_Shape: ", y.shape) #3, 1, 48
#     print(y.size(1))
    assert d == y.size(1)

    x = x.unsqueeze(1).expand(n, m, d)
#     print(x)
#     print("x_u: ", x.shape)
    y = y.unsqueeze(0).expand(n, m, d)
#     print(y)
#     print("y_u: ", y.shape)
    return torch.pow(x - y, 2).sum(2)

In [20]:

# query = request.FILES.get('query', None).name
# s1 = request.FILES.get('s1', None).name
# audio.append(s1)
# s2 = request.FILES.get('s2', None).name
# audio.append(s2)
# s3 = request.FILES.get('s3', None).name
# audio.append(s3)
# s4 = request.FILES.get('s4', None).name
# audio.append(s4)

def get_support(audio):
    support = {}
    labels = []
    for file in audio:
        label = file.split("_")[0]
        if label not in labels:
            labels.append(label)
        if label not in support:
            support[label] = []
        for same in audio:
            if label == same.split("_")[0]:
                if same not in support[label]:
                    support[label].append(same)
    return support, labels

def predictLabel(query, support, labels):
    x, label = getData(query)
    xq_emb = model.encoder(x.reshape(1, 1, 16, 40))
#     print("xq_emb: ", xq_emb.shape)
    su_label = []
    su_pro = []
    for key, value in support.items():
        su_label.append(key)
        tmp = []
        check = 0
        count = 0
        for i in value:
            x, label_i = getData(i)
            feature = x.reshape(1, 1, 16, 40)
            emb = model.encoder.forward(feature).squeeze()
#             print("y_emb:", emb.shape)
            check += emb
            tmp.append(emb.detach().numpy())
            count += 1
#         print("check:", (check/3).shape)
#         print("check:", check/count, count)
        tmp = np.array(tmp)
        pro = (tmp).mean(0)
#         print("pro: ", pro)
        su_pro.append(pro)
#     dists = euclidean_dist(xq_emb.reshape(1, 48), torch.tensor(su_pro))
    dists = euclidean_dist(xq_emb.reshape(1, 9), torch.tensor(su_pro))
    n_class, n_query = torch.tensor(su_pro).shape[0], 1
    log_p_y = F.log_softmax(-dists, dim=1).view(n_class, n_query, -1)
#     print("log_py: ", log_p_y)
    g, y_hat = log_p_y.max(0)
#     print("log_py: ", log_p_y.max(0))
#     print("g, y_hat: ", g, y_hat)
    pred = labels[y_hat.squeeze()]
    return pred, label

In [5]:
path = "D:\\HoangNgan\\ThucTap\\Cough_data_Tuan1\\mono"
label_list = []
label_dict = {}
for i in listdir(path):
    if ".txt" not in i and 'aug' not in i:
        if 'whooping' not in i:
            label_list.append(i)
            label_dict[str(i)] = len(listdir(os.path.join(path, i)))
            print(str(i) + ": ", len(listdir(os.path.join(path, i))))
        else:
            label_list.append(i)
            label_dict[str(i)] = 60
            print(str(i) + ": ", label_dict[str(i)] )
    

barkingcough:  1040
chestyandwetcough:  1663
coughingupcrapagain:  884
dryafternooncough:  1040
gaggywetcough:  1117
heavycoldandsorethroatcoughing:  416
nightwetcough:  1300
springallergycoughing:  884
unknown:  1557
whoopingcough:  60


In [5]:
label_dict[label_list[1]]

1663

In [6]:
print(label_list)

['barkingcough', 'chestyandwetcough', 'coughingupcrapagain', 'dryafternooncough', 'gaggywetcough', 'heavycoldandsorethroatcoughing', 'nightwetcough', 'springallergycoughing', 'unknown', 'whoopingcough']


In [7]:
audio = []
query = "whoopingcough_30.wav"
# audio.append(query)
s1 = "unknown_1.wav"
audio.append(s1)
s2 = "unknown_1.wav"
audio.append(s2)
s3 = "unknown_11.wav"
audio.append(s3)
s4 = "unknown_21.wav"
audio.append(s4)

In [6]:
support, labels = get_support(audio)
print(support, labels)
print(predictLabel(query, support, labels))

NameError: name 'audio' is not defined

In [6]:
#setup dataset
#2way-2shot
#train
train_label = label_list[:8]
test_label = label_list[8:]

In [7]:
print(test_label)

['unknown', 'whoopingcough']


In [72]:
check_data_unk = {'0': ['unknown_1.wav', 'unknown_635.wav', 'unknown_960.wav', 'whoopingcough_5.wav', 'whoopingcough_37.wav'], '1': ['unknown_2.wav', 'unknown_394.wav', 'unknown_154.wav', 'whoopingcough_37.wav', 'whoopingcough_51.wav'], '2': ['unknown_3.wav', 'unknown_769.wav', 'unknown_568.wav', 'whoopingcough_60.wav', 'whoopingcough_22.wav'], '3': ['unknown_4.wav', 'unknown_143.wav', 'unknown_646.wav', 'whoopingcough_35.wav', 'whoopingcough_4.wav'], '4': ['unknown_5.wav', 'unknown_1551.wav', 'unknown_1500.wav', 'whoopingcough_14.wav', 'whoopingcough_48.wav'], '5': ['unknown_6.wav', 'unknown_1440.wav', 'unknown_1439.wav', 'whoopingcough_41.wav', 'whoopingcough_58.wav'], '6': ['unknown_7.wav', 'unknown_133.wav', 'unknown_752.wav', 'whoopingcough_51.wav', 'whoopingcough_52.wav'], '7': ['unknown_8.wav', 'unknown_1363.wav', 'unknown_65.wav', 'whoopingcough_37.wav', 'whoopingcough_3.wav'], '8': ['unknown_9.wav', 'unknown_753.wav', 'unknown_1540.wav', 'whoopingcough_28.wav', 'whoopingcough_32.wav'], '9': ['unknown_10.wav', 'unknown_181.wav', 'unknown_1100.wav', 'whoopingcough_18.wav', 'whoopingcough_46.wav'], '10': ['unknown_11.wav', 'unknown_1250.wav', 'unknown_551.wav', 'whoopingcough_33.wav', 'whoopingcough_1.wav'], '11': ['unknown_12.wav', 'unknown_240.wav', 'unknown_972.wav', 'whoopingcough_3.wav', 'whoopingcough_20.wav'], '12': ['unknown_13.wav', 'unknown_270.wav', 'unknown_1140.wav', 'whoopingcough_45.wav', 'whoopingcough_54.wav'], '13': ['unknown_14.wav', 'unknown_1127.wav', 'unknown_41.wav', 'whoopingcough_14.wav', 'whoopingcough_20.wav'], '14': ['unknown_15.wav', 'unknown_1358.wav', 'unknown_859.wav', 'whoopingcough_32.wav', 'whoopingcough_9.wav'], '15': ['unknown_16.wav', 'unknown_624.wav', 'unknown_258.wav', 'whoopingcough_41.wav', 'whoopingcough_52.wav'], '16': ['unknown_17.wav', 'unknown_1353.wav', 'unknown_794.wav', 'whoopingcough_54.wav', 'whoopingcough_43.wav'], '17': ['unknown_18.wav', 'unknown_669.wav', 'unknown_408.wav', 'whoopingcough_36.wav', 'whoopingcough_15.wav'], '18': ['unknown_19.wav', 'unknown_1410.wav', 'unknown_974.wav', 'whoopingcough_12.wav', 'whoopingcough_26.wav'], '19': ['unknown_20.wav', 'unknown_342.wav', 'unknown_1309.wav', 'whoopingcough_49.wav', 'whoopingcough_35.wav'], '20': ['unknown_21.wav', 'unknown_1528.wav', 'unknown_743.wav', 'whoopingcough_19.wav', 'whoopingcough_4.wav'], '21': ['unknown_22.wav', 'unknown_1256.wav', 'unknown_1193.wav', 'whoopingcough_40.wav', 'whoopingcough_60.wav'], '22': ['unknown_23.wav', 'unknown_1448.wav', 'unknown_545.wav', 'whoopingcough_36.wav', 'whoopingcough_47.wav'], '23': ['unknown_24.wav', 'unknown_1078.wav', 'unknown_1512.wav', 'whoopingcough_35.wav', 'whoopingcough_38.wav'], '24': ['unknown_25.wav', 'unknown_923.wav', 'unknown_1071.wav', 'whoopingcough_55.wav', 'whoopingcough_52.wav'], '25': ['unknown_26.wav', 'unknown_467.wav', 'unknown_1138.wav', 'whoopingcough_43.wav', 'whoopingcough_40.wav'], '26': ['unknown_27.wav', 'unknown_434.wav', 'unknown_222.wav', 'whoopingcough_22.wav', 'whoopingcough_21.wav'], '27': ['unknown_28.wav', 'unknown_241.wav', 'unknown_867.wav', 'whoopingcough_21.wav', 'whoopingcough_58.wav'], '28': ['unknown_29.wav', 'unknown_94.wav', 'unknown_1250.wav', 'whoopingcough_31.wav', 'whoopingcough_4.wav'], '29': ['unknown_30.wav', 'unknown_478.wav', 'unknown_51.wav', 'whoopingcough_10.wav', 'whoopingcough_18.wav'], '30': ['unknown_31.wav', 'unknown_1253.wav', 'unknown_1179.wav', 'whoopingcough_18.wav', 'whoopingcough_46.wav'], '31': ['unknown_32.wav', 'unknown_415.wav', 'unknown_536.wav', 'whoopingcough_40.wav', 'whoopingcough_19.wav'], '32': ['unknown_33.wav', 'unknown_1078.wav', 'unknown_235.wav', 'whoopingcough_28.wav', 'whoopingcough_56.wav'], '33': ['unknown_34.wav', 'unknown_1549.wav', 'unknown_534.wav', 'whoopingcough_7.wav', 'whoopingcough_12.wav'], '34': ['unknown_35.wav', 'unknown_1256.wav', 'unknown_747.wav', 'whoopingcough_13.wav', 'whoopingcough_14.wav'], '35': ['unknown_36.wav', 'unknown_1137.wav', 'unknown_727.wav', 'whoopingcough_41.wav', 'whoopingcough_28.wav'], '36': ['unknown_37.wav', 'unknown_1230.wav', 'unknown_490.wav', 'whoopingcough_5.wav', 'whoopingcough_53.wav'], '37': ['unknown_38.wav', 'unknown_885.wav', 'unknown_1095.wav', 'whoopingcough_46.wav', 'whoopingcough_55.wav'], '38': ['unknown_39.wav', 'unknown_1008.wav', 'unknown_660.wav', 'whoopingcough_8.wav', 'whoopingcough_36.wav'], '39': ['unknown_40.wav', 'unknown_358.wav', 'unknown_83.wav', 'whoopingcough_60.wav', 'whoopingcough_12.wav'], '40': ['unknown_41.wav', 'unknown_1514.wav', 'unknown_699.wav', 'whoopingcough_47.wav', 'whoopingcough_38.wav'], '41': ['unknown_42.wav', 'unknown_678.wav', 'unknown_1061.wav', 'whoopingcough_21.wav', 'whoopingcough_27.wav'], '42': ['unknown_43.wav', 'unknown_1416.wav', 'unknown_390.wav', 'whoopingcough_4.wav', 'whoopingcough_24.wav'], '43': ['unknown_44.wav', 'unknown_970.wav', 'unknown_759.wav', 'whoopingcough_37.wav', 'whoopingcough_25.wav'], '44': ['unknown_45.wav', 'unknown_581.wav', 'unknown_29.wav', 'whoopingcough_57.wav', 'whoopingcough_1.wav'], '45': ['unknown_46.wav', 'unknown_482.wav', 'unknown_602.wav', 'whoopingcough_32.wav', 'whoopingcough_31.wav'], '46': ['unknown_47.wav', 'unknown_1266.wav', 'unknown_588.wav', 'whoopingcough_8.wav', 'whoopingcough_49.wav'], '47': ['unknown_48.wav', 'unknown_1003.wav', 'unknown_1318.wav', 'whoopingcough_44.wav', 'whoopingcough_46.wav'], '48': ['unknown_49.wav', 'unknown_358.wav', 'unknown_1010.wav', 'whoopingcough_41.wav', 'whoopingcough_54.wav'], '49': ['unknown_50.wav', 'unknown_1549.wav', 'unknown_1023.wav', 'whoopingcough_10.wav', 'whoopingcough_24.wav'], '50': ['unknown_51.wav', 'unknown_380.wav', 'unknown_417.wav', 'whoopingcough_60.wav', 'whoopingcough_52.wav'], '51': ['unknown_52.wav', 'unknown_669.wav', 'unknown_33.wav', 'whoopingcough_11.wav', 'whoopingcough_50.wav'], '52': ['unknown_53.wav', 'unknown_8.wav', 'unknown_840.wav', 'whoopingcough_15.wav', 'whoopingcough_40.wav'], '53': ['unknown_54.wav', 'unknown_1261.wav', 'unknown_665.wav', 'whoopingcough_33.wav', 'whoopingcough_7.wav'], '54': ['unknown_55.wav', 'unknown_1542.wav', 'unknown_442.wav', 'whoopingcough_30.wav', 'whoopingcough_58.wav'], '55': ['unknown_56.wav', 'unknown_1030.wav', 'unknown_1154.wav', 'whoopingcough_5.wav', 'whoopingcough_48.wav'], '56': ['unknown_57.wav', 'unknown_981.wav', 'unknown_524.wav', 'whoopingcough_6.wav', 'whoopingcough_1.wav'], '57': ['unknown_58.wav', 'unknown_1028.wav', 'unknown_531.wav', 'whoopingcough_26.wav', 'whoopingcough_46.wav'], '58': ['unknown_59.wav', 'unknown_285.wav', 'unknown_454.wav', 'whoopingcough_14.wav', 'whoopingcough_35.wav'], '59': ['unknown_60.wav', 'unknown_603.wav', 'unknown_1003.wav', 'whoopingcough_6.wav', 'whoopingcough_56.wav']}

In [73]:
check_data_wh = {'0': ['whoopingcough_1.wav', 'whoopingcough_18.wav', 'whoopingcough_27.wav', 'unknown_660.wav', 'unknown_144.wav'], '1': ['whoopingcough_2.wav', 'whoopingcough_6.wav', 'whoopingcough_30.wav', 'unknown_299.wav', 'unknown_1260.wav'], '2': ['whoopingcough_3.wav', 'whoopingcough_15.wav', 'whoopingcough_4.wav', 'unknown_554.wav', 'unknown_739.wav'], '3': ['whoopingcough_4.wav', 'whoopingcough_34.wav', 'whoopingcough_28.wav', 'unknown_1324.wav', 'unknown_188.wav'], '4': ['whoopingcough_5.wav', 'whoopingcough_52.wav', 'whoopingcough_47.wav', 'unknown_1438.wav', 'unknown_196.wav'], '5': ['whoopingcough_6.wav', 'whoopingcough_3.wav', 'whoopingcough_11.wav', 'unknown_900.wav', 'unknown_675.wav'], '6': ['whoopingcough_7.wav', 'whoopingcough_19.wav', 'whoopingcough_32.wav', 'unknown_933.wav', 'unknown_1198.wav'], '7': ['whoopingcough_8.wav', 'whoopingcough_13.wav', 'whoopingcough_45.wav', 'unknown_558.wav', 'unknown_1230.wav'], '8': ['whoopingcough_9.wav', 'whoopingcough_11.wav', 'whoopingcough_50.wav', 'unknown_708.wav', 'unknown_1462.wav'], '9': ['whoopingcough_10.wav', 'whoopingcough_16.wav', 'whoopingcough_54.wav', 'unknown_335.wav', 'unknown_783.wav'], '10': ['whoopingcough_11.wav', 'whoopingcough_35.wav', 'whoopingcough_10.wav', 'unknown_831.wav', 'unknown_729.wav'], '11': ['whoopingcough_12.wav', 'whoopingcough_11.wav', 'whoopingcough_56.wav', 'unknown_234.wav', 'unknown_1450.wav'], '12': ['whoopingcough_13.wav', 'whoopingcough_16.wav', 'whoopingcough_40.wav', 'unknown_1403.wav', 'unknown_1234.wav'], '13': ['whoopingcough_14.wav', 'whoopingcough_28.wav', 'whoopingcough_56.wav', 'unknown_516.wav', 'unknown_475.wav'], '14': ['whoopingcough_15.wav', 'whoopingcough_14.wav', 'whoopingcough_18.wav', 'unknown_145.wav', 'unknown_1063.wav'], '15': ['whoopingcough_16.wav', 'whoopingcough_51.wav', 'whoopingcough_49.wav', 'unknown_267.wav', 'unknown_1003.wav'], '16': ['whoopingcough_17.wav', 'whoopingcough_24.wav', 'whoopingcough_57.wav', 'unknown_165.wav', 'unknown_1190.wav'], '17': ['whoopingcough_18.wav', 'whoopingcough_60.wav', 'whoopingcough_35.wav', 'unknown_1039.wav', 'unknown_125.wav'], '18': ['whoopingcough_19.wav', 'whoopingcough_36.wav', 'whoopingcough_58.wav', 'unknown_678.wav', 'unknown_1118.wav'], '19': ['whoopingcough_20.wav', 'whoopingcough_18.wav', 'whoopingcough_15.wav', 'unknown_887.wav', 'unknown_792.wav'], '20': ['whoopingcough_21.wav', 'whoopingcough_18.wav', 'whoopingcough_17.wav', 'unknown_107.wav', 'unknown_569.wav'], '21': ['whoopingcough_22.wav', 'whoopingcough_52.wav', 'whoopingcough_28.wav', 'unknown_117.wav', 'unknown_114.wav'], '22': ['whoopingcough_23.wav', 'whoopingcough_17.wav', 'whoopingcough_40.wav', 'unknown_23.wav', 'unknown_40.wav'], '23': ['whoopingcough_24.wav', 'whoopingcough_6.wav', 'whoopingcough_18.wav', 'unknown_52.wav', 'unknown_264.wav'], '24': ['whoopingcough_25.wav', 'whoopingcough_52.wav', 'whoopingcough_38.wav', 'unknown_1423.wav', 'unknown_548.wav'], '25': ['whoopingcough_26.wav', 'whoopingcough_3.wav', 'whoopingcough_24.wav', 'unknown_213.wav', 'unknown_386.wav'], '26': ['whoopingcough_27.wav', 'whoopingcough_59.wav', 'whoopingcough_24.wav', 'unknown_1288.wav', 'unknown_115.wav'], '27': ['whoopingcough_28.wav', 'whoopingcough_3.wav', 'whoopingcough_60.wav', 'unknown_948.wav', 'unknown_137.wav'], '28': ['whoopingcough_29.wav', 'whoopingcough_44.wav', 'whoopingcough_32.wav', 'unknown_141.wav', 'unknown_1134.wav'], '29': ['whoopingcough_30.wav', 'whoopingcough_36.wav', 'whoopingcough_51.wav', 'unknown_597.wav', 'unknown_841.wav'], '30': ['whoopingcough_31.wav', 'whoopingcough_51.wav', 'whoopingcough_49.wav', 'unknown_316.wav', 'unknown_978.wav'], '31': ['whoopingcough_32.wav', 'whoopingcough_28.wav', 'whoopingcough_51.wav', 'unknown_764.wav', 'unknown_141.wav'], '32': ['whoopingcough_33.wav', 'whoopingcough_53.wav', 'whoopingcough_4.wav', 'unknown_1528.wav', 'unknown_390.wav'], '33': ['whoopingcough_34.wav', 'whoopingcough_14.wav', 'whoopingcough_10.wav', 'unknown_1435.wav', 'unknown_1192.wav'], '34': ['whoopingcough_35.wav', 'whoopingcough_28.wav', 'whoopingcough_49.wav', 'unknown_393.wav', 'unknown_110.wav'], '35': ['whoopingcough_36.wav', 'whoopingcough_13.wav', 'whoopingcough_31.wav', 'unknown_282.wav', 'unknown_27.wav'], '36': ['whoopingcough_37.wav', 'whoopingcough_26.wav', 'whoopingcough_23.wav', 'unknown_951.wav', 'unknown_497.wav'], '37': ['whoopingcough_38.wav', 'whoopingcough_50.wav', 'whoopingcough_19.wav', 'unknown_1220.wav', 'unknown_1089.wav'], '38': ['whoopingcough_39.wav', 'whoopingcough_50.wav', 'whoopingcough_28.wav', 'unknown_1366.wav', 'unknown_1367.wav'], '39': ['whoopingcough_40.wav', 'whoopingcough_20.wav', 'whoopingcough_30.wav', 'unknown_811.wav', 'unknown_420.wav'], '40': ['whoopingcough_41.wav', 'whoopingcough_46.wav', 'whoopingcough_7.wav', 'unknown_750.wav', 'unknown_576.wav'], '41': ['whoopingcough_42.wav', 'whoopingcough_3.wav', 'whoopingcough_55.wav', 'unknown_1064.wav', 'unknown_1268.wav'], '42': ['whoopingcough_43.wav', 'whoopingcough_40.wav', 'whoopingcough_55.wav', 'unknown_233.wav', 'unknown_410.wav'], '43': ['whoopingcough_44.wav', 'whoopingcough_3.wav', 'whoopingcough_60.wav', 'unknown_570.wav', 'unknown_974.wav'], '44': ['whoopingcough_45.wav', 'whoopingcough_26.wav', 'whoopingcough_9.wav', 'unknown_44.wav', 'unknown_821.wav'], '45': ['whoopingcough_46.wav', 'whoopingcough_3.wav', 'whoopingcough_51.wav', 'unknown_320.wav', 'unknown_492.wav'], '46': ['whoopingcough_47.wav', 'whoopingcough_50.wav', 'whoopingcough_32.wav', 'unknown_156.wav', 'unknown_10.wav'], '47': ['whoopingcough_48.wav', 'whoopingcough_29.wav', 'whoopingcough_58.wav', 'unknown_1072.wav', 'unknown_701.wav'], '48': ['whoopingcough_49.wav', 'whoopingcough_41.wav', 'whoopingcough_40.wav', 'unknown_1005.wav', 'unknown_1000.wav'], '49': ['whoopingcough_50.wav', 'whoopingcough_5.wav', 'whoopingcough_25.wav', 'unknown_120.wav', 'unknown_1409.wav'], '50': ['whoopingcough_51.wav', 'whoopingcough_7.wav', 'whoopingcough_31.wav', 'unknown_875.wav', 'unknown_1231.wav'], '51': ['whoopingcough_52.wav', 'whoopingcough_5.wav', 'whoopingcough_60.wav', 'unknown_13.wav', 'unknown_344.wav'], '52': ['whoopingcough_53.wav', 'whoopingcough_22.wav', 'whoopingcough_10.wav', 'unknown_1556.wav', 'unknown_679.wav'], '53': ['whoopingcough_54.wav', 'whoopingcough_2.wav', 'whoopingcough_33.wav', 'unknown_488.wav', 'unknown_1340.wav'], '54': ['whoopingcough_55.wav', 'whoopingcough_13.wav', 'whoopingcough_58.wav', 'unknown_1055.wav', 'unknown_128.wav'], '55': ['whoopingcough_56.wav', 'whoopingcough_40.wav', 'whoopingcough_9.wav', 'unknown_1110.wav', 'unknown_165.wav'], '56': ['whoopingcough_57.wav', 'whoopingcough_2.wav', 'whoopingcough_24.wav', 'unknown_1423.wav', 'unknown_1394.wav'], '57': ['whoopingcough_58.wav', 'whoopingcough_6.wav', 'whoopingcough_32.wav', 'unknown_31.wav', 'unknown_1125.wav'], '58': ['whoopingcough_59.wav', 'whoopingcough_52.wav', 'whoopingcough_12.wav', 'unknown_463.wav', 'unknown_1110.wav'], '59': ['whoopingcough_60.wav', 'whoopingcough_14.wav', 'whoopingcough_17.wav', 'unknown_1305.wav', 'unknown_1435.wav']}

In [8]:
#label1:
import random
def set_support_label(index_label):
    index_list = list(range(len(train_label)))
#     print(index_label)
    index_list.remove(index_label)
#     print(index_list)
    new_index_list = index_list
    ix = random.choice(new_index_list)
    label1 = train_label[index_label]
    label2 = train_label[ix]
    return index_label, ix, label1, label2


def set_support_label2(index_label):
    index_list = list(range(len(test_label)))
#     print(index_label)
    index_list.remove(index_label)
#     print(index_list)
    new_index_list = index_list
    ix = random.choice(new_index_list)
    label1 = test_label[index_label]
    label2 = test_label[ix]
    return index_label, ix, label1, label2

def set_support_data(label, index_audio):
#     if index_label == 0:
#         index_label = 8
#     else:
#         index_label = 9
    index_list = list(range(1, label_dict[label]+1))
#     index = random.choice(index_list)
    index_list.remove(index_audio)
    new_index_list = index_list
    ix = random.choice(new_index_list)
    
    new_index_list.remove(ix)
    temp_list = new_index_list
    ixx = random.choice(temp_list)
    audio1 = label+'_'+str(index_audio)+'.wav'
    audio2 = label+'_'+str(ix)+'.wav'
    audio3 = label+'_'+str(ixx)+'.wav'
    return audio1, audio2, audio3

print(set_support_label(0))
# print(list(range(len(train_label))))
print(set_support_data("whoopingcough", 32))

(0, 3, 'barkingcough', 'dryafternooncough')
('whoopingcough_32.wav', 'whoopingcough_25.wav', 'whoopingcough_31.wav')


In [14]:
query1, support1, labels1 = None, None, None

In [9]:
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import classification_report, confusion_matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1, keepdims = True)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [16]:
data = ['a', 'b', 'c']
data.remove(data[1])
print(data)

['a', 'c']


In [10]:
#PREPARE DATA
from numpy.random import choice
cough_dataset = '/content/drive/MyDrive/cough_detection/cough_data/mono_test'
cough_dataset = "D:\\HoangNgan\\ThucTap\\Cough_data_Tuan1\\mono"

def sup_que(num_sup, que, sup):
    '''create few-shot tasks: 
   a few-shot task: [query, same_label_support1, same_label_support2, different_label_support1, different_label_support2]
   this function creates 60 tasks for each label'''
    check_data = {}
    
    
    class_set =  os.path.join(cough_dataset, que)
    data = os.listdir(class_set)
    len_data = len(data)
#     print(len_data)
    count = 0
    for index in range(60):
#         print(index)
        data = os.listdir(class_set)
        i = 0
        query = data[index]
        query_set = []
        query_set.append(query)
        data.remove(data[index])
#         print(index)
        support_set = []
        query_list = choice(data,size=(num_sup),replace=False)
        for q in query_list:
            query_set.append(q)
        

        i = i+1
        class_set_next =  os.path.join(cough_dataset, sup)
        data_next = os.listdir(class_set_next)
        support_set = choice(data_next,size=(num_sup),replace=False)

        check_data[str(count)] = list(query_set) + list(support_set)
        count += 1
    # if count == 5:
    #   temp = que
    #   que = sup
    #   sup = temp

  # print(check_data)
    return check_data
  

    

In [11]:
train_data = ['springallergycoughing',
'dryafternooncough',
'coughingupcrapagain',
'gaggywetcough',
'chestyandwetcough',
'barkingcough',
'nightwetcough',
'heavycoldandsorethroatcoughing']
test_data = ['whoopingcough', 'unknown']

In [19]:
way2shot2_unk= sup_que(2, test_data[1], test_data[0])
way2shot2_wh = sup_que(2, test_data[0], test_data[1])

In [22]:
unk = []
wh = []
for i in range(5):
    way2shot2_unk= sup_que(2, test_data[1], test_data[0])
    way2shot2_wh = sup_que(2, test_data[0], test_data[1])
    unk.append(way2shot2_unk)
    wh.append(way2shot2_wh)

In [59]:
count = 0
for i in range(60):
#     print(unk[0][str(i)])
#     print(unk[2][str(i)])
#     print('---------')
    if wh[3][str(i)][1:] == wh[4][str(i)][1:]:
        count += 1
print('count: ', count)
    

count:  0


In [48]:
print(unk)

[{'0': ['unknown_1.wav', 'unknown_1248.wav', 'unknown_1040.wav', 'whoopingcough_41.wav', 'whoopingcough_36.wav'], '1': ['unknown_10.wav', 'unknown_869.wav', 'unknown_384.wav', 'whoopingcough_55.wav', 'whoopingcough_33.wav'], '2': ['unknown_100.wav', 'unknown_933.wav', 'unknown_1089.wav', 'whoopingcough_44.wav', 'whoopingcough_16.wav'], '3': ['unknown_1000.wav', 'unknown_40.wav', 'unknown_17.wav', 'whoopingcough_12.wav', 'whoopingcough_54.wav'], '4': ['unknown_1001.wav', 'unknown_1019.wav', 'unknown_362.wav', 'whoopingcough_7.wav', 'whoopingcough_32.wav'], '5': ['unknown_1002.wav', 'unknown_773.wav', 'unknown_1416.wav', 'whoopingcough_25.wav', 'whoopingcough_21.wav'], '6': ['unknown_1003.wav', 'unknown_1491.wav', 'unknown_463.wav', 'whoopingcough_36.wav', 'whoopingcough_13.wav'], '7': ['unknown_1004.wav', 'unknown_34.wav', 'unknown_441.wav', 'whoopingcough_14.wav', 'whoopingcough_39.wav'], '8': ['unknown_1005.wav', 'unknown_257.wav', 'unknown_1330.wav', 'whoopingcough_5.wav', 'whoopingc

In [12]:
unk = [{'0': ['unknown_1.wav', 'unknown_1248.wav', 'unknown_1040.wav', 'whoopingcough_41.wav', 'whoopingcough_36.wav'], '1': ['unknown_10.wav', 'unknown_869.wav', 'unknown_384.wav', 'whoopingcough_55.wav', 'whoopingcough_33.wav'], '2': ['unknown_100.wav', 'unknown_933.wav', 'unknown_1089.wav', 'whoopingcough_44.wav', 'whoopingcough_16.wav'], '3': ['unknown_1000.wav', 'unknown_40.wav', 'unknown_17.wav', 'whoopingcough_12.wav', 'whoopingcough_54.wav'], '4': ['unknown_1001.wav', 'unknown_1019.wav', 'unknown_362.wav', 'whoopingcough_7.wav', 'whoopingcough_32.wav'], '5': ['unknown_1002.wav', 'unknown_773.wav', 'unknown_1416.wav', 'whoopingcough_25.wav', 'whoopingcough_21.wav'], '6': ['unknown_1003.wav', 'unknown_1491.wav', 'unknown_463.wav', 'whoopingcough_36.wav', 'whoopingcough_13.wav'], '7': ['unknown_1004.wav', 'unknown_34.wav', 'unknown_441.wav', 'whoopingcough_14.wav', 'whoopingcough_39.wav'], '8': ['unknown_1005.wav', 'unknown_257.wav', 'unknown_1330.wav', 'whoopingcough_5.wav', 'whoopingcough_55.wav'], '9': ['unknown_1006.wav', 'unknown_949.wav', 'unknown_474.wav', 'whoopingcough_38.wav', 'whoopingcough_41.wav'], '10': ['unknown_1007.wav', 'unknown_993.wav', 'unknown_821.wav', 'whoopingcough_45.wav', 'whoopingcough_12.wav'], '11': ['unknown_1008.wav', 'unknown_1382.wav', 'unknown_930.wav', 'whoopingcough_47.wav', 'whoopingcough_42.wav'], '12': ['unknown_1009.wav', 'unknown_1245.wav', 'unknown_1268.wav', 'whoopingcough_60.wav', 'whoopingcough_2.wav'], '13': ['unknown_101.wav', 'unknown_1469.wav', 'unknown_1448.wav', 'whoopingcough_34.wav', 'whoopingcough_19.wav'], '14': ['unknown_1010.wav', 'unknown_2.wav', 'unknown_505.wav', 'whoopingcough_48.wav', 'whoopingcough_10.wav'], '15': ['unknown_1011.wav', 'unknown_1193.wav', 'unknown_689.wav', 'whoopingcough_43.wav', 'whoopingcough_2.wav'], '16': ['unknown_1012.wav', 'unknown_821.wav', 'unknown_1303.wav', 'whoopingcough_23.wav', 'whoopingcough_8.wav'], '17': ['unknown_1013.wav', 'unknown_1416.wav', 'unknown_1319.wav', 'whoopingcough_27.wav', 'whoopingcough_22.wav'], '18': ['unknown_1014.wav', 'unknown_1288.wav', 'unknown_761.wav', 'whoopingcough_36.wav', 'whoopingcough_23.wav'], '19': ['unknown_1015.wav', 'unknown_1336.wav', 'unknown_834.wav', 'whoopingcough_29.wav', 'whoopingcough_36.wav'], '20': ['unknown_1016.wav', 'unknown_1358.wav', 'unknown_74.wav', 'whoopingcough_2.wav', 'whoopingcough_48.wav'], '21': ['unknown_1017.wav', 'unknown_1542.wav', 'unknown_922.wav', 'whoopingcough_41.wav', 'whoopingcough_21.wav'], '22': ['unknown_1018.wav', 'unknown_603.wav', 'unknown_876.wav', 'whoopingcough_6.wav', 'whoopingcough_2.wav'], '23': ['unknown_1019.wav', 'unknown_62.wav', 'unknown_133.wav', 'whoopingcough_53.wav', 'whoopingcough_26.wav'], '24': ['unknown_102.wav', 'unknown_323.wav', 'unknown_1200.wav', 'whoopingcough_49.wav', 'whoopingcough_54.wav'], '25': ['unknown_1020.wav', 'unknown_1204.wav', 'unknown_1000.wav', 'whoopingcough_15.wav', 'whoopingcough_21.wav'], '26': ['unknown_1021.wav', 'unknown_577.wav', 'unknown_1285.wav', 'whoopingcough_37.wav', 'whoopingcough_54.wav'], '27': ['unknown_1022.wav', 'unknown_812.wav', 'unknown_576.wav', 'whoopingcough_46.wav', 'whoopingcough_49.wav'], '28': ['unknown_1023.wav', 'unknown_1408.wav', 'unknown_1332.wav', 'whoopingcough_49.wav', 'whoopingcough_8.wav'], '29': ['unknown_1024.wav', 'unknown_821.wav', 'unknown_1544.wav', 'whoopingcough_31.wav', 'whoopingcough_29.wav'], '30': ['unknown_1025.wav', 'unknown_738.wav', 'unknown_784.wav', 'whoopingcough_2.wav', 'whoopingcough_35.wav'], '31': ['unknown_1026.wav', 'unknown_1196.wav', 'unknown_1295.wav', 'whoopingcough_33.wav', 'whoopingcough_58.wav'], '32': ['unknown_1027.wav', 'unknown_1498.wav', 'unknown_1076.wav', 'whoopingcough_53.wav', 'whoopingcough_30.wav'], '33': ['unknown_1028.wav', 'unknown_267.wav', 'unknown_1271.wav', 'whoopingcough_32.wav', 'whoopingcough_12.wav'], '34': ['unknown_1029.wav', 'unknown_433.wav', 'unknown_413.wav', 'whoopingcough_47.wav', 'whoopingcough_10.wav'], '35': ['unknown_103.wav', 'unknown_251.wav', 'unknown_1219.wav', 'whoopingcough_36.wav', 'whoopingcough_8.wav'], '36': ['unknown_1030.wav', 'unknown_1198.wav', 'unknown_107.wav', 'whoopingcough_24.wav', 'whoopingcough_41.wav'], '37': ['unknown_1031.wav', 'unknown_778.wav', 'unknown_1466.wav', 'whoopingcough_31.wav', 'whoopingcough_54.wav'], '38': ['unknown_1032.wav', 'unknown_1501.wav', 'unknown_975.wav', 'whoopingcough_9.wav', 'whoopingcough_2.wav'], '39': ['unknown_1033.wav', 'unknown_707.wav', 'unknown_832.wav', 'whoopingcough_17.wav', 'whoopingcough_51.wav'], '40': ['unknown_1034.wav', 'unknown_1556.wav', 'unknown_999.wav', 'whoopingcough_28.wav', 'whoopingcough_49.wav'], '41': ['unknown_1035.wav', 'unknown_78.wav', 'unknown_1485.wav', 'whoopingcough_18.wav', 'whoopingcough_43.wav'], '42': ['unknown_1036.wav', 'unknown_958.wav', 'unknown_704.wav', 'whoopingcough_59.wav', 'whoopingcough_55.wav'], '43': ['unknown_1037.wav', 'unknown_1295.wav', 'unknown_333.wav', 'whoopingcough_42.wav', 'whoopingcough_59.wav'], '44': ['unknown_1038.wav', 'unknown_1157.wav', 'unknown_1542.wav', 'whoopingcough_7.wav', 'whoopingcough_47.wav'], '45': ['unknown_1039.wav', 'unknown_1545.wav', 'unknown_1297.wav', 'whoopingcough_40.wav', 'whoopingcough_53.wav'], '46': ['unknown_104.wav', 'unknown_1498.wav', 'unknown_23.wav', 'whoopingcough_47.wav', 'whoopingcough_52.wav'], '47': ['unknown_1040.wav', 'unknown_78.wav', 'unknown_230.wav', 'whoopingcough_28.wav', 'whoopingcough_39.wav'], '48': ['unknown_1041.wav', 'unknown_896.wav', 'unknown_1037.wav', 'whoopingcough_48.wav', 'whoopingcough_47.wav'], '49': ['unknown_1042.wav', 'unknown_660.wav', 'unknown_606.wav', 'whoopingcough_18.wav', 'whoopingcough_44.wav'], '50': ['unknown_1043.wav', 'unknown_655.wav', 'unknown_1320.wav', 'whoopingcough_53.wav', 'whoopingcough_35.wav'], '51': ['unknown_1044.wav', 'unknown_202.wav', 'unknown_127.wav', 'whoopingcough_35.wav', 'whoopingcough_56.wav'], '52': ['unknown_1045.wav', 'unknown_1482.wav', 'unknown_584.wav', 'whoopingcough_8.wav', 'whoopingcough_31.wav'], '53': ['unknown_1046.wav', 'unknown_1449.wav', 'unknown_884.wav', 'whoopingcough_50.wav', 'whoopingcough_18.wav'], '54': ['unknown_1047.wav', 'unknown_516.wav', 'unknown_929.wav', 'whoopingcough_53.wav', 'whoopingcough_12.wav'], '55': ['unknown_1048.wav', 'unknown_628.wav', 'unknown_817.wav', 'whoopingcough_35.wav', 'whoopingcough_50.wav'], '56': ['unknown_1049.wav', 'unknown_648.wav', 'unknown_224.wav', 'whoopingcough_11.wav', 'whoopingcough_9.wav'], '57': ['unknown_105.wav', 'unknown_73.wav', 'unknown_979.wav', 'whoopingcough_28.wav', 'whoopingcough_4.wav'], '58': ['unknown_1050.wav', 'unknown_996.wav', 'unknown_1332.wav', 'whoopingcough_39.wav', 'whoopingcough_6.wav'], '59': ['unknown_1051.wav', 'unknown_1243.wav', 'unknown_1134.wav', 'whoopingcough_8.wav', 'whoopingcough_46.wav']}, {'0': ['unknown_1.wav', 'unknown_1549.wav', 'unknown_1180.wav', 'whoopingcough_41.wav', 'whoopingcough_9.wav'], '1': ['unknown_10.wav', 'unknown_721.wav', 'unknown_919.wav', 'whoopingcough_59.wav', 'whoopingcough_18.wav'], '2': ['unknown_100.wav', 'unknown_1530.wav', 'unknown_161.wav', 'whoopingcough_39.wav', 'whoopingcough_56.wav'], '3': ['unknown_1000.wav', 'unknown_617.wav', 'unknown_127.wav', 'whoopingcough_53.wav', 'whoopingcough_22.wav'], '4': ['unknown_1001.wav', 'unknown_1310.wav', 'unknown_433.wav', 'whoopingcough_51.wav', 'whoopingcough_36.wav'], '5': ['unknown_1002.wav', 'unknown_991.wav', 'unknown_1030.wav', 'whoopingcough_14.wav', 'whoopingcough_3.wav'], '6': ['unknown_1003.wav', 'unknown_514.wav', 'unknown_441.wav', 'whoopingcough_14.wav', 'whoopingcough_21.wav'], '7': ['unknown_1004.wav', 'unknown_1377.wav', 'unknown_1001.wav', 'whoopingcough_42.wav', 'whoopingcough_9.wav'], '8': ['unknown_1005.wav', 'unknown_109.wav', 'unknown_551.wav', 'whoopingcough_28.wav', 'whoopingcough_23.wav'], '9': ['unknown_1006.wav', 'unknown_1552.wav', 'unknown_1057.wav', 'whoopingcough_9.wav', 'whoopingcough_29.wav'], '10': ['unknown_1007.wav', 'unknown_655.wav', 'unknown_1293.wav', 'whoopingcough_45.wav', 'whoopingcough_41.wav'], '11': ['unknown_1008.wav', 'unknown_941.wav', 'unknown_1464.wav', 'whoopingcough_48.wav', 'whoopingcough_31.wav'], '12': ['unknown_1009.wav', 'unknown_970.wav', 'unknown_1105.wav', 'whoopingcough_58.wav', 'whoopingcough_55.wav'], '13': ['unknown_101.wav', 'unknown_1367.wav', 'unknown_1350.wav', 'whoopingcough_15.wav', 'whoopingcough_3.wav'], '14': ['unknown_1010.wav', 'unknown_1323.wav', 'unknown_294.wav', 'whoopingcough_54.wav', 'whoopingcough_48.wav'], '15': ['unknown_1011.wav', 'unknown_845.wav', 'unknown_711.wav', 'whoopingcough_30.wav', 'whoopingcough_12.wav'], '16': ['unknown_1012.wav', 'unknown_232.wav', 'unknown_1229.wav', 'whoopingcough_26.wav', 'whoopingcough_13.wav'], '17': ['unknown_1013.wav', 'unknown_1061.wav', 'unknown_807.wav', 'whoopingcough_46.wav', 'whoopingcough_10.wav'], '18': ['unknown_1014.wav', 'unknown_170.wav', 'unknown_314.wav', 'whoopingcough_7.wav', 'whoopingcough_50.wav'], '19': ['unknown_1015.wav', 'unknown_1269.wav', 'unknown_1331.wav', 'whoopingcough_25.wav', 'whoopingcough_29.wav'], '20': ['unknown_1016.wav', 'unknown_517.wav', 'unknown_547.wav', 'whoopingcough_41.wav', 'whoopingcough_24.wav'], '21': ['unknown_1017.wav', 'unknown_764.wav', 'unknown_1526.wav', 'whoopingcough_57.wav', 'whoopingcough_55.wav'], '22': ['unknown_1018.wav', 'unknown_454.wav', 'unknown_37.wav', 'whoopingcough_52.wav', 'whoopingcough_49.wav'], '23': ['unknown_1019.wav', 'unknown_225.wav', 'unknown_603.wav', 'whoopingcough_14.wav', 'whoopingcough_33.wav'], '24': ['unknown_102.wav', 'unknown_900.wav', 'unknown_894.wav', 'whoopingcough_15.wav', 'whoopingcough_49.wav'], '25': ['unknown_1020.wav', 'unknown_1405.wav', 'unknown_901.wav', 'whoopingcough_7.wav', 'whoopingcough_37.wav'], '26': ['unknown_1021.wav', 'unknown_130.wav', 'unknown_1118.wav', 'whoopingcough_49.wav', 'whoopingcough_24.wav'], '27': ['unknown_1022.wav', 'unknown_11.wav', 'unknown_848.wav', 'whoopingcough_14.wav', 'whoopingcough_50.wav'], '28': ['unknown_1023.wav', 'unknown_714.wav', 'unknown_1387.wav', 'whoopingcough_59.wav', 'whoopingcough_28.wav'], '29': ['unknown_1024.wav', 'unknown_1168.wav', 'unknown_761.wav', 'whoopingcough_39.wav', 'whoopingcough_50.wav'], '30': ['unknown_1025.wav', 'unknown_1205.wav', 'unknown_1211.wav', 'whoopingcough_12.wav', 'whoopingcough_8.wav'], '31': ['unknown_1026.wav', 'unknown_1311.wav', 'unknown_1307.wav', 'whoopingcough_44.wav', 'whoopingcough_14.wav'], '32': ['unknown_1027.wav', 'unknown_1343.wav', 'unknown_117.wav', 'whoopingcough_22.wav', 'whoopingcough_11.wav'], '33': ['unknown_1028.wav', 'unknown_1134.wav', 'unknown_1080.wav', 'whoopingcough_25.wav', 'whoopingcough_54.wav'], '34': ['unknown_1029.wav', 'unknown_876.wav', 'unknown_1249.wav', 'whoopingcough_41.wav', 'whoopingcough_56.wav'], '35': ['unknown_103.wav', 'unknown_1252.wav', 'unknown_215.wav', 'whoopingcough_34.wav', 'whoopingcough_9.wav'], '36': ['unknown_1030.wav', 'unknown_1009.wav', 'unknown_1065.wav', 'whoopingcough_10.wav', 'whoopingcough_54.wav'], '37': ['unknown_1031.wav', 'unknown_1135.wav', 'unknown_650.wav', 'whoopingcough_17.wav', 'whoopingcough_53.wav'], '38': ['unknown_1032.wav', 'unknown_511.wav', 'unknown_1223.wav', 'whoopingcough_26.wav', 'whoopingcough_49.wav'], '39': ['unknown_1033.wav', 'unknown_8.wav', 'unknown_373.wav', 'whoopingcough_52.wav', 'whoopingcough_46.wav'], '40': ['unknown_1034.wav', 'unknown_168.wav', 'unknown_1009.wav', 'whoopingcough_30.wav', 'whoopingcough_37.wav'], '41': ['unknown_1035.wav', 'unknown_767.wav', 'unknown_747.wav', 'whoopingcough_53.wav', 'whoopingcough_23.wav'], '42': ['unknown_1036.wav', 'unknown_1323.wav', 'unknown_684.wav', 'whoopingcough_20.wav', 'whoopingcough_25.wav'], '43': ['unknown_1037.wav', 'unknown_213.wav', 'unknown_714.wav', 'whoopingcough_9.wav', 'whoopingcough_28.wav'], '44': ['unknown_1038.wav', 'unknown_1307.wav', 'unknown_663.wav', 'whoopingcough_18.wav', 'whoopingcough_54.wav'], '45': ['unknown_1039.wav', 'unknown_1261.wav', 'unknown_914.wav', 'whoopingcough_24.wav', 'whoopingcough_13.wav'], '46': ['unknown_104.wav', 'unknown_1311.wav', 'unknown_294.wav', 'whoopingcough_14.wav', 'whoopingcough_42.wav'], '47': ['unknown_1040.wav', 'unknown_1435.wav', 'unknown_487.wav', 'whoopingcough_13.wav', 'whoopingcough_12.wav'], '48': ['unknown_1041.wav', 'unknown_1385.wav', 'unknown_378.wav', 'whoopingcough_14.wav', 'whoopingcough_43.wav'], '49': ['unknown_1042.wav', 'unknown_681.wav', 'unknown_132.wav', 'whoopingcough_43.wav', 'whoopingcough_45.wav'], '50': ['unknown_1043.wav', 'unknown_546.wav', 'unknown_1407.wav', 'whoopingcough_44.wav', 'whoopingcough_45.wav'], '51': ['unknown_1044.wav', 'unknown_630.wav', 'unknown_11.wav', 'whoopingcough_29.wav', 'whoopingcough_47.wav'], '52': ['unknown_1045.wav', 'unknown_1295.wav', 'unknown_550.wav', 'whoopingcough_19.wav', 'whoopingcough_14.wav'], '53': ['unknown_1046.wav', 'unknown_1095.wav', 'unknown_1342.wav', 'whoopingcough_33.wav', 'whoopingcough_50.wav'], '54': ['unknown_1047.wav', 'unknown_1491.wav', 'unknown_691.wav', 'whoopingcough_1.wav', 'whoopingcough_44.wav'], '55': ['unknown_1048.wav', 'unknown_482.wav', 'unknown_641.wav', 'whoopingcough_6.wav', 'whoopingcough_24.wav'], '56': ['unknown_1049.wav', 'unknown_895.wav', 'unknown_24.wav', 'whoopingcough_55.wav', 'whoopingcough_36.wav'], '57': ['unknown_105.wav', 'unknown_525.wav', 'unknown_1402.wav', 'whoopingcough_18.wav', 'whoopingcough_58.wav'], '58': ['unknown_1050.wav', 'unknown_998.wav', 'unknown_1254.wav', 'whoopingcough_15.wav', 'whoopingcough_44.wav'], '59': ['unknown_1051.wav', 'unknown_59.wav', 'unknown_1012.wav', 'whoopingcough_40.wav', 'whoopingcough_45.wav']}, {'0': ['unknown_1.wav', 'unknown_430.wav', 'unknown_1308.wav', 'whoopingcough_57.wav', 'whoopingcough_5.wav'], '1': ['unknown_10.wav', 'unknown_865.wav', 'unknown_252.wav', 'whoopingcough_9.wav', 'whoopingcough_44.wav'], '2': ['unknown_100.wav', 'unknown_138.wav', 'unknown_1556.wav', 'whoopingcough_53.wav', 'whoopingcough_40.wav'], '3': ['unknown_1000.wav', 'unknown_777.wav', 'unknown_83.wav', 'whoopingcough_1.wav', 'whoopingcough_18.wav'], '4': ['unknown_1001.wav', 'unknown_222.wav', 'unknown_674.wav', 'whoopingcough_48.wav', 'whoopingcough_34.wav'], '5': ['unknown_1002.wav', 'unknown_566.wav', 'unknown_909.wav', 'whoopingcough_36.wav', 'whoopingcough_2.wav'], '6': ['unknown_1003.wav', 'unknown_133.wav', 'unknown_884.wav', 'whoopingcough_18.wav', 'whoopingcough_41.wav'], '7': ['unknown_1004.wav', 'unknown_877.wav', 'unknown_1055.wav', 'whoopingcough_12.wav', 'whoopingcough_21.wav'], '8': ['unknown_1005.wav', 'unknown_1520.wav', 'unknown_949.wav', 'whoopingcough_48.wav', 'whoopingcough_29.wav'], '9': ['unknown_1006.wav', 'unknown_1044.wav', 'unknown_548.wav', 'whoopingcough_32.wav', 'whoopingcough_57.wav'], '10': ['unknown_1007.wav', 'unknown_379.wav', 'unknown_533.wav', 'whoopingcough_10.wav', 'whoopingcough_16.wav'], '11': ['unknown_1008.wav', 'unknown_717.wav', 'unknown_1055.wav', 'whoopingcough_19.wav', 'whoopingcough_38.wav'], '12': ['unknown_1009.wav', 'unknown_1504.wav', 'unknown_899.wav', 'whoopingcough_17.wav', 'whoopingcough_56.wav'], '13': ['unknown_101.wav', 'unknown_619.wav', 'unknown_471.wav', 'whoopingcough_57.wav', 'whoopingcough_2.wav'], '14': ['unknown_1010.wav', 'unknown_695.wav', 'unknown_826.wav', 'whoopingcough_42.wav', 'whoopingcough_45.wav'], '15': ['unknown_1011.wav', 'unknown_1289.wav', 'unknown_619.wav', 'whoopingcough_40.wav', 'whoopingcough_20.wav'], '16': ['unknown_1012.wav', 'unknown_439.wav', 'unknown_223.wav', 'whoopingcough_36.wav', 'whoopingcough_43.wav'], '17': ['unknown_1013.wav', 'unknown_106.wav', 'unknown_569.wav', 'whoopingcough_59.wav', 'whoopingcough_46.wav'], '18': ['unknown_1014.wav', 'unknown_442.wav', 'unknown_94.wav', 'whoopingcough_44.wav', 'whoopingcough_6.wav'], '19': ['unknown_1015.wav', 'unknown_677.wav', 'unknown_1236.wav', 'whoopingcough_30.wav', 'whoopingcough_58.wav'], '20': ['unknown_1016.wav', 'unknown_1377.wav', 'unknown_64.wav', 'whoopingcough_29.wav', 'whoopingcough_11.wav'], '21': ['unknown_1017.wav', 'unknown_188.wav', 'unknown_890.wav', 'whoopingcough_18.wav', 'whoopingcough_1.wav'], '22': ['unknown_1018.wav', 'unknown_1324.wav', 'unknown_1073.wav', 'whoopingcough_49.wav', 'whoopingcough_7.wav'], '23': ['unknown_1019.wav', 'unknown_725.wav', 'unknown_1222.wav', 'whoopingcough_38.wav', 'whoopingcough_7.wav'], '24': ['unknown_102.wav', 'unknown_1010.wav', 'unknown_1389.wav', 'whoopingcough_5.wav', 'whoopingcough_21.wav'], '25': ['unknown_1020.wav', 'unknown_664.wav', 'unknown_16.wav', 'whoopingcough_7.wav', 'whoopingcough_4.wav'], '26': ['unknown_1021.wav', 'unknown_1359.wav', 'unknown_1114.wav', 'whoopingcough_19.wav', 'whoopingcough_10.wav'], '27': ['unknown_1022.wav', 'unknown_352.wav', 'unknown_320.wav', 'whoopingcough_38.wav', 'whoopingcough_49.wav'], '28': ['unknown_1023.wav', 'unknown_213.wav', 'unknown_1360.wav', 'whoopingcough_20.wav', 'whoopingcough_41.wav'], '29': ['unknown_1024.wav', 'unknown_422.wav', 'unknown_1103.wav', 'whoopingcough_44.wav', 'whoopingcough_1.wav'], '30': ['unknown_1025.wav', 'unknown_747.wav', 'unknown_485.wav', 'whoopingcough_14.wav', 'whoopingcough_60.wav'], '31': ['unknown_1026.wav', 'unknown_1542.wav', 'unknown_1090.wav', 'whoopingcough_54.wav', 'whoopingcough_46.wav'], '32': ['unknown_1027.wav', 'unknown_477.wav', 'unknown_748.wav', 'whoopingcough_23.wav', 'whoopingcough_25.wav'], '33': ['unknown_1028.wav', 'unknown_380.wav', 'unknown_364.wav', 'whoopingcough_2.wav', 'whoopingcough_18.wav'], '34': ['unknown_1029.wav', 'unknown_1106.wav', 'unknown_892.wav', 'whoopingcough_42.wav', 'whoopingcough_22.wav'], '35': ['unknown_103.wav', 'unknown_162.wav', 'unknown_294.wav', 'whoopingcough_54.wav', 'whoopingcough_38.wav'], '36': ['unknown_1030.wav', 'unknown_1440.wav', 'unknown_1302.wav', 'whoopingcough_53.wav', 'whoopingcough_21.wav'], '37': ['unknown_1031.wav', 'unknown_940.wav', 'unknown_296.wav', 'whoopingcough_19.wav', 'whoopingcough_59.wav'], '38': ['unknown_1032.wav', 'unknown_173.wav', 'unknown_1503.wav', 'whoopingcough_36.wav', 'whoopingcough_46.wav'], '39': ['unknown_1033.wav', 'unknown_515.wav', 'unknown_155.wav', 'whoopingcough_39.wav', 'whoopingcough_43.wav'], '40': ['unknown_1034.wav', 'unknown_89.wav', 'unknown_1194.wav', 'whoopingcough_55.wav', 'whoopingcough_51.wav'], '41': ['unknown_1035.wav', 'unknown_1117.wav', 'unknown_536.wav', 'whoopingcough_13.wav', 'whoopingcough_40.wav'], '42': ['unknown_1036.wav', 'unknown_269.wav', 'unknown_567.wav', 'whoopingcough_26.wav', 'whoopingcough_10.wav'], '43': ['unknown_1037.wav', 'unknown_214.wav', 'unknown_1459.wav', 'whoopingcough_30.wav', 'whoopingcough_4.wav'], '44': ['unknown_1038.wav', 'unknown_1375.wav', 'unknown_1166.wav', 'whoopingcough_57.wav', 'whoopingcough_21.wav'], '45': ['unknown_1039.wav', 'unknown_763.wav', 'unknown_1309.wav', 'whoopingcough_16.wav', 'whoopingcough_52.wav'], '46': ['unknown_104.wav', 'unknown_891.wav', 'unknown_1503.wav', 'whoopingcough_48.wav', 'whoopingcough_1.wav'], '47': ['unknown_1040.wav', 'unknown_1423.wav', 'unknown_314.wav', 'whoopingcough_6.wav', 'whoopingcough_16.wav'], '48': ['unknown_1041.wav', 'unknown_1075.wav', 'unknown_431.wav', 'whoopingcough_15.wav', 'whoopingcough_18.wav'], '49': ['unknown_1042.wav', 'unknown_990.wav', 'unknown_77.wav', 'whoopingcough_26.wav', 'whoopingcough_46.wav'], '50': ['unknown_1043.wav', 'unknown_830.wav', 'unknown_1221.wav', 'whoopingcough_53.wav', 'whoopingcough_16.wav'], '51': ['unknown_1044.wav', 'unknown_246.wav', 'unknown_1425.wav', 'whoopingcough_12.wav', 'whoopingcough_16.wav'], '52': ['unknown_1045.wav', 'unknown_1521.wav', 'unknown_356.wav', 'whoopingcough_26.wav', 'whoopingcough_22.wav'], '53': ['unknown_1046.wav', 'unknown_249.wav', 'unknown_422.wav', 'whoopingcough_1.wav', 'whoopingcough_15.wav'], '54': ['unknown_1047.wav', 'unknown_1005.wav', 'unknown_677.wav', 'whoopingcough_34.wav', 'whoopingcough_58.wav'], '55': ['unknown_1048.wav', 'unknown_1199.wav', 'unknown_1062.wav', 'whoopingcough_46.wav', 'whoopingcough_44.wav'], '56': ['unknown_1049.wav', 'unknown_740.wav', 'unknown_1118.wav', 'whoopingcough_5.wav', 'whoopingcough_9.wav'], '57': ['unknown_105.wav', 'unknown_1284.wav', 'unknown_1428.wav', 'whoopingcough_45.wav', 'whoopingcough_15.wav'], '58': ['unknown_1050.wav', 'unknown_194.wav', 'unknown_140.wav', 'whoopingcough_23.wav', 'whoopingcough_38.wav'], '59': ['unknown_1051.wav', 'unknown_1070.wav', 'unknown_876.wav', 'whoopingcough_47.wav', 'whoopingcough_22.wav']}, {'0': ['unknown_1.wav', 'unknown_811.wav', 'unknown_572.wav', 'whoopingcough_43.wav', 'whoopingcough_17.wav'], '1': ['unknown_10.wav', 'unknown_220.wav', 'unknown_714.wav', 'whoopingcough_12.wav', 'whoopingcough_30.wav'], '2': ['unknown_100.wav', 'unknown_485.wav', 'unknown_221.wav', 'whoopingcough_24.wav', 'whoopingcough_4.wav'], '3': ['unknown_1000.wav', 'unknown_343.wav', 'unknown_296.wav', 'whoopingcough_2.wav', 'whoopingcough_55.wav'], '4': ['unknown_1001.wav', 'unknown_566.wav', 'unknown_1033.wav', 'whoopingcough_42.wav', 'whoopingcough_11.wav'], '5': ['unknown_1002.wav', 'unknown_1466.wav', 'unknown_908.wav', 'whoopingcough_44.wav', 'whoopingcough_58.wav'], '6': ['unknown_1003.wav', 'unknown_1504.wav', 'unknown_1238.wav', 'whoopingcough_7.wav', 'whoopingcough_55.wav'], '7': ['unknown_1004.wav', 'unknown_1352.wav', 'unknown_861.wav', 'whoopingcough_32.wav', 'whoopingcough_15.wav'], '8': ['unknown_1005.wav', 'unknown_244.wav', 'unknown_1345.wav', 'whoopingcough_39.wav', 'whoopingcough_55.wav'], '9': ['unknown_1006.wav', 'unknown_538.wav', 'unknown_351.wav', 'whoopingcough_55.wav', 'whoopingcough_17.wav'], '10': ['unknown_1007.wav', 'unknown_1346.wav', 'unknown_187.wav', 'whoopingcough_20.wav', 'whoopingcough_34.wav'], '11': ['unknown_1008.wav', 'unknown_1079.wav', 'unknown_22.wav', 'whoopingcough_29.wav', 'whoopingcough_5.wav'], '12': ['unknown_1009.wav', 'unknown_1393.wav', 'unknown_657.wav', 'whoopingcough_13.wav', 'whoopingcough_6.wav'], '13': ['unknown_101.wav', 'unknown_1235.wav', 'unknown_452.wav', 'whoopingcough_33.wav', 'whoopingcough_34.wav'], '14': ['unknown_1010.wav', 'unknown_376.wav', 'unknown_812.wav', 'whoopingcough_36.wav', 'whoopingcough_13.wav'], '15': ['unknown_1011.wav', 'unknown_87.wav', 'unknown_1062.wav', 'whoopingcough_49.wav', 'whoopingcough_12.wav'], '16': ['unknown_1012.wav', 'unknown_664.wav', 'unknown_902.wav', 'whoopingcough_30.wav', 'whoopingcough_46.wav'], '17': ['unknown_1013.wav', 'unknown_223.wav', 'unknown_587.wav', 'whoopingcough_35.wav', 'whoopingcough_45.wav'], '18': ['unknown_1014.wav', 'unknown_1342.wav', 'unknown_1474.wav', 'whoopingcough_2.wav', 'whoopingcough_25.wav'], '19': ['unknown_1015.wav', 'unknown_1306.wav', 'unknown_1155.wav', 'whoopingcough_35.wav', 'whoopingcough_8.wav'], '20': ['unknown_1016.wav', 'unknown_473.wav', 'unknown_1066.wav', 'whoopingcough_60.wav', 'whoopingcough_42.wav'], '21': ['unknown_1017.wav', 'unknown_600.wav', 'unknown_1532.wav', 'whoopingcough_24.wav', 'whoopingcough_19.wav'], '22': ['unknown_1018.wav', 'unknown_1326.wav', 'unknown_899.wav', 'whoopingcough_11.wav', 'whoopingcough_10.wav'], '23': ['unknown_1019.wav', 'unknown_765.wav', 'unknown_1011.wav', 'whoopingcough_42.wav', 'whoopingcough_49.wav'], '24': ['unknown_102.wav', 'unknown_682.wav', 'unknown_882.wav', 'whoopingcough_45.wav', 'whoopingcough_14.wav'], '25': ['unknown_1020.wav', 'unknown_11.wav', 'unknown_778.wav', 'whoopingcough_56.wav', 'whoopingcough_27.wav'], '26': ['unknown_1021.wav', 'unknown_32.wav', 'unknown_812.wav', 'whoopingcough_17.wav', 'whoopingcough_6.wav'], '27': ['unknown_1022.wav', 'unknown_1424.wav', 'unknown_380.wav', 'whoopingcough_21.wav', 'whoopingcough_2.wav'], '28': ['unknown_1023.wav', 'unknown_111.wav', 'unknown_688.wav', 'whoopingcough_39.wav', 'whoopingcough_22.wav'], '29': ['unknown_1024.wav', 'unknown_1538.wav', 'unknown_873.wav', 'whoopingcough_43.wav', 'whoopingcough_29.wav'], '30': ['unknown_1025.wav', 'unknown_889.wav', 'unknown_176.wav', 'whoopingcough_9.wav', 'whoopingcough_45.wav'], '31': ['unknown_1026.wav', 'unknown_1263.wav', 'unknown_1200.wav', 'whoopingcough_3.wav', 'whoopingcough_59.wav'], '32': ['unknown_1027.wav', 'unknown_717.wav', 'unknown_1083.wav', 'whoopingcough_57.wav', 'whoopingcough_16.wav'], '33': ['unknown_1028.wav', 'unknown_680.wav', 'unknown_1260.wav', 'whoopingcough_36.wav', 'whoopingcough_3.wav'], '34': ['unknown_1029.wav', 'unknown_942.wav', 'unknown_953.wav', 'whoopingcough_13.wav', 'whoopingcough_10.wav'], '35': ['unknown_103.wav', 'unknown_797.wav', 'unknown_1134.wav', 'whoopingcough_18.wav', 'whoopingcough_5.wav'], '36': ['unknown_1030.wav', 'unknown_110.wav', 'unknown_1453.wav', 'whoopingcough_38.wav', 'whoopingcough_52.wav'], '37': ['unknown_1031.wav', 'unknown_1522.wav', 'unknown_1549.wav', 'whoopingcough_38.wav', 'whoopingcough_6.wav'], '38': ['unknown_1032.wav', 'unknown_1507.wav', 'unknown_1437.wav', 'whoopingcough_5.wav', 'whoopingcough_9.wav'], '39': ['unknown_1033.wav', 'unknown_884.wav', 'unknown_1370.wav', 'whoopingcough_25.wav', 'whoopingcough_11.wav'], '40': ['unknown_1034.wav', 'unknown_39.wav', 'unknown_364.wav', 'whoopingcough_15.wav', 'whoopingcough_38.wav'], '41': ['unknown_1035.wav', 'unknown_556.wav', 'unknown_925.wav', 'whoopingcough_16.wav', 'whoopingcough_55.wav'], '42': ['unknown_1036.wav', 'unknown_1119.wav', 'unknown_651.wav', 'whoopingcough_4.wav', 'whoopingcough_17.wav'], '43': ['unknown_1037.wav', 'unknown_181.wav', 'unknown_560.wav', 'whoopingcough_30.wav', 'whoopingcough_8.wav'], '44': ['unknown_1038.wav', 'unknown_1271.wav', 'unknown_1018.wav', 'whoopingcough_10.wav', 'whoopingcough_60.wav'], '45': ['unknown_1039.wav', 'unknown_1181.wav', 'unknown_949.wav', 'whoopingcough_50.wav', 'whoopingcough_7.wav'], '46': ['unknown_104.wav', 'unknown_337.wav', 'unknown_1196.wav', 'whoopingcough_14.wav', 'whoopingcough_38.wav'], '47': ['unknown_1040.wav', 'unknown_1341.wav', 'unknown_1062.wav', 'whoopingcough_4.wav', 'whoopingcough_46.wav'], '48': ['unknown_1041.wav', 'unknown_115.wav', 'unknown_864.wav', 'whoopingcough_5.wav', 'whoopingcough_54.wav'], '49': ['unknown_1042.wav', 'unknown_1177.wav', 'unknown_37.wav', 'whoopingcough_26.wav', 'whoopingcough_18.wav'], '50': ['unknown_1043.wav', 'unknown_171.wav', 'unknown_1230.wav', 'whoopingcough_49.wav', 'whoopingcough_60.wav'], '51': ['unknown_1044.wav', 'unknown_482.wav', 'unknown_442.wav', 'whoopingcough_55.wav', 'whoopingcough_28.wav'], '52': ['unknown_1045.wav', 'unknown_1530.wav', 'unknown_565.wav', 'whoopingcough_24.wav', 'whoopingcough_8.wav'], '53': ['unknown_1046.wav', 'unknown_529.wav', 'unknown_1383.wav', 'whoopingcough_7.wav', 'whoopingcough_43.wav'], '54': ['unknown_1047.wav', 'unknown_1288.wav', 'unknown_130.wav', 'whoopingcough_36.wav', 'whoopingcough_52.wav'], '55': ['unknown_1048.wav', 'unknown_206.wav', 'unknown_614.wav', 'whoopingcough_20.wav', 'whoopingcough_10.wav'], '56': ['unknown_1049.wav', 'unknown_973.wav', 'unknown_559.wav', 'whoopingcough_29.wav', 'whoopingcough_51.wav'], '57': ['unknown_105.wav', 'unknown_92.wav', 'unknown_396.wav', 'whoopingcough_48.wav', 'whoopingcough_34.wav'], '58': ['unknown_1050.wav', 'unknown_200.wav', 'unknown_31.wav', 'whoopingcough_28.wav', 'whoopingcough_5.wav'], '59': ['unknown_1051.wav', 'unknown_627.wav', 'unknown_1179.wav', 'whoopingcough_21.wav', 'whoopingcough_22.wav']}, {'0': ['unknown_1.wav', 'unknown_611.wav', 'unknown_266.wav', 'whoopingcough_35.wav', 'whoopingcough_4.wav'], '1': ['unknown_10.wav', 'unknown_131.wav', 'unknown_1029.wav', 'whoopingcough_24.wav', 'whoopingcough_59.wav'], '2': ['unknown_100.wav', 'unknown_516.wav', 'unknown_1500.wav', 'whoopingcough_15.wav', 'whoopingcough_12.wav'], '3': ['unknown_1000.wav', 'unknown_186.wav', 'unknown_1426.wav', 'whoopingcough_59.wav', 'whoopingcough_50.wav'], '4': ['unknown_1001.wav', 'unknown_354.wav', 'unknown_293.wav', 'whoopingcough_56.wav', 'whoopingcough_47.wav'], '5': ['unknown_1002.wav', 'unknown_808.wav', 'unknown_958.wav', 'whoopingcough_23.wav', 'whoopingcough_24.wav'], '6': ['unknown_1003.wav', 'unknown_28.wav', 'unknown_1503.wav', 'whoopingcough_48.wav', 'whoopingcough_16.wav'], '7': ['unknown_1004.wav', 'unknown_366.wav', 'unknown_1128.wav', 'whoopingcough_49.wav', 'whoopingcough_55.wav'], '8': ['unknown_1005.wav', 'unknown_632.wav', 'unknown_836.wav', 'whoopingcough_49.wav', 'whoopingcough_8.wav'], '9': ['unknown_1006.wav', 'unknown_302.wav', 'unknown_1387.wav', 'whoopingcough_54.wav', 'whoopingcough_41.wav'], '10': ['unknown_1007.wav', 'unknown_1196.wav', 'unknown_565.wav', 'whoopingcough_31.wav', 'whoopingcough_54.wav'], '11': ['unknown_1008.wav', 'unknown_546.wav', 'unknown_896.wav', 'whoopingcough_37.wav', 'whoopingcough_29.wav'], '12': ['unknown_1009.wav', 'unknown_1279.wav', 'unknown_910.wav', 'whoopingcough_60.wav', 'whoopingcough_28.wav'], '13': ['unknown_101.wav', 'unknown_1369.wav', 'unknown_502.wav', 'whoopingcough_53.wav', 'whoopingcough_38.wav'], '14': ['unknown_1010.wav', 'unknown_850.wav', 'unknown_1435.wav', 'whoopingcough_55.wav', 'whoopingcough_30.wav'], '15': ['unknown_1011.wav', 'unknown_744.wav', 'unknown_1554.wav', 'whoopingcough_31.wav', 'whoopingcough_11.wav'], '16': ['unknown_1012.wav', 'unknown_253.wav', 'unknown_3.wav', 'whoopingcough_7.wav', 'whoopingcough_27.wav'], '17': ['unknown_1013.wav', 'unknown_1110.wav', 'unknown_124.wav', 'whoopingcough_44.wav', 'whoopingcough_15.wav'], '18': ['unknown_1014.wav', 'unknown_1385.wav', 'unknown_7.wav', 'whoopingcough_38.wav', 'whoopingcough_2.wav'], '19': ['unknown_1015.wav', 'unknown_798.wav', 'unknown_220.wav', 'whoopingcough_26.wav', 'whoopingcough_41.wav'], '20': ['unknown_1016.wav', 'unknown_1339.wav', 'unknown_1220.wav', 'whoopingcough_45.wav', 'whoopingcough_39.wav'], '21': ['unknown_1017.wav', 'unknown_1243.wav', 'unknown_1193.wav', 'whoopingcough_47.wav', 'whoopingcough_40.wav'], '22': ['unknown_1018.wav', 'unknown_549.wav', 'unknown_623.wav', 'whoopingcough_4.wav', 'whoopingcough_12.wav'], '23': ['unknown_1019.wav', 'unknown_887.wav', 'unknown_48.wav', 'whoopingcough_41.wav', 'whoopingcough_17.wav'], '24': ['unknown_102.wav', 'unknown_221.wav', 'unknown_446.wav', 'whoopingcough_32.wav', 'whoopingcough_13.wav'], '25': ['unknown_1020.wav', 'unknown_191.wav', 'unknown_861.wav', 'whoopingcough_46.wav', 'whoopingcough_41.wav'], '26': ['unknown_1021.wav', 'unknown_274.wav', 'unknown_906.wav', 'whoopingcough_44.wav', 'whoopingcough_26.wav'], '27': ['unknown_1022.wav', 'unknown_1308.wav', 'unknown_205.wav', 'whoopingcough_16.wav', 'whoopingcough_49.wav'], '28': ['unknown_1023.wav', 'unknown_1137.wav', 'unknown_1101.wav', 'whoopingcough_60.wav', 'whoopingcough_52.wav'], '29': ['unknown_1024.wav', 'unknown_183.wav', 'unknown_722.wav', 'whoopingcough_16.wav', 'whoopingcough_28.wav'], '30': ['unknown_1025.wav', 'unknown_397.wav', 'unknown_704.wav', 'whoopingcough_27.wav', 'whoopingcough_55.wav'], '31': ['unknown_1026.wav', 'unknown_827.wav', 'unknown_822.wav', 'whoopingcough_9.wav', 'whoopingcough_26.wav'], '32': ['unknown_1027.wav', 'unknown_807.wav', 'unknown_911.wav', 'whoopingcough_52.wav', 'whoopingcough_6.wav'], '33': ['unknown_1028.wav', 'unknown_314.wav', 'unknown_289.wav', 'whoopingcough_50.wav', 'whoopingcough_60.wav'], '34': ['unknown_1029.wav', 'unknown_643.wav', 'unknown_1127.wav', 'whoopingcough_42.wav', 'whoopingcough_25.wav'], '35': ['unknown_103.wav', 'unknown_1520.wav', 'unknown_1037.wav', 'whoopingcough_11.wav', 'whoopingcough_17.wav'], '36': ['unknown_1030.wav', 'unknown_1219.wav', 'unknown_754.wav', 'whoopingcough_59.wav', 'whoopingcough_10.wav'], '37': ['unknown_1031.wav', 'unknown_446.wav', 'unknown_283.wav', 'whoopingcough_28.wav', 'whoopingcough_8.wav'], '38': ['unknown_1032.wav', 'unknown_231.wav', 'unknown_420.wav', 'whoopingcough_4.wav', 'whoopingcough_29.wav'], '39': ['unknown_1033.wav', 'unknown_490.wav', 'unknown_835.wav', 'whoopingcough_3.wav', 'whoopingcough_37.wav'], '40': ['unknown_1034.wav', 'unknown_1206.wav', 'unknown_1491.wav', 'whoopingcough_47.wav', 'whoopingcough_11.wav'], '41': ['unknown_1035.wav', 'unknown_1013.wav', 'unknown_848.wav', 'whoopingcough_38.wav', 'whoopingcough_42.wav'], '42': ['unknown_1036.wav', 'unknown_936.wav', 'unknown_1437.wav', 'whoopingcough_7.wav', 'whoopingcough_18.wav'], '43': ['unknown_1037.wav', 'unknown_1531.wav', 'unknown_750.wav', 'whoopingcough_51.wav', 'whoopingcough_25.wav'], '44': ['unknown_1038.wav', 'unknown_1298.wav', 'unknown_467.wav', 'whoopingcough_10.wav', 'whoopingcough_9.wav'], '45': ['unknown_1039.wav', 'unknown_81.wav', 'unknown_1364.wav', 'whoopingcough_40.wav', 'whoopingcough_35.wav'], '46': ['unknown_104.wav', 'unknown_1251.wav', 'unknown_773.wav', 'whoopingcough_3.wav', 'whoopingcough_12.wav'], '47': ['unknown_1040.wav', 'unknown_765.wav', 'unknown_1257.wav', 'whoopingcough_11.wav', 'whoopingcough_44.wav'], '48': ['unknown_1041.wav', 'unknown_949.wav', 'unknown_133.wav', 'whoopingcough_53.wav', 'whoopingcough_23.wav'], '49': ['unknown_1042.wav', 'unknown_1495.wav', 'unknown_1500.wav', 'whoopingcough_23.wav', 'whoopingcough_47.wav'], '50': ['unknown_1043.wav', 'unknown_570.wav', 'unknown_1216.wav', 'whoopingcough_37.wav', 'whoopingcough_31.wav'], '51': ['unknown_1044.wav', 'unknown_37.wav', 'unknown_1476.wav', 'whoopingcough_40.wav', 'whoopingcough_43.wav'], '52': ['unknown_1045.wav', 'unknown_361.wav', 'unknown_1269.wav', 'whoopingcough_32.wav', 'whoopingcough_35.wav'], '53': ['unknown_1046.wav', 'unknown_484.wav', 'unknown_979.wav', 'whoopingcough_52.wav', 'whoopingcough_51.wav'], '54': ['unknown_1047.wav', 'unknown_49.wav', 'unknown_55.wav', 'whoopingcough_43.wav', 'whoopingcough_4.wav'], '55': ['unknown_1048.wav', 'unknown_1539.wav', 'unknown_1551.wav', 'whoopingcough_25.wav', 'whoopingcough_1.wav'], '56': ['unknown_1049.wav', 'unknown_1061.wav', 'unknown_1080.wav', 'whoopingcough_36.wav', 'whoopingcough_27.wav'], '57': ['unknown_105.wav', 'unknown_270.wav', 'unknown_349.wav', 'whoopingcough_14.wav', 'whoopingcough_16.wav'], '58': ['unknown_1050.wav', 'unknown_213.wav', 'unknown_1528.wav', 'whoopingcough_50.wav', 'whoopingcough_20.wav'], '59': ['unknown_1051.wav', 'unknown_926.wav', 'unknown_308.wav', 'whoopingcough_23.wav', 'whoopingcough_49.wav']}]

In [60]:
print(wh)

[{'0': ['whoopingcough_1.wav', 'whoopingcough_8.wav', 'whoopingcough_41.wav', 'unknown_403.wav', 'unknown_324.wav'], '1': ['whoopingcough_10.wav', 'whoopingcough_39.wav', 'whoopingcough_5.wav', 'unknown_583.wav', 'unknown_1186.wav'], '2': ['whoopingcough_11.wav', 'whoopingcough_45.wav', 'whoopingcough_50.wav', 'unknown_272.wav', 'unknown_823.wav'], '3': ['whoopingcough_12.wav', 'whoopingcough_48.wav', 'whoopingcough_26.wav', 'unknown_285.wav', 'unknown_311.wav'], '4': ['whoopingcough_13.wav', 'whoopingcough_52.wav', 'whoopingcough_33.wav', 'unknown_16.wav', 'unknown_1334.wav'], '5': ['whoopingcough_14.wav', 'whoopingcough_45.wav', 'whoopingcough_33.wav', 'unknown_784.wav', 'unknown_1081.wav'], '6': ['whoopingcough_15.wav', 'whoopingcough_48.wav', 'whoopingcough_40.wav', 'unknown_707.wav', 'unknown_478.wav'], '7': ['whoopingcough_16.wav', 'whoopingcough_57.wav', 'whoopingcough_54.wav', 'unknown_1489.wav', 'unknown_235.wav'], '8': ['whoopingcough_17.wav', 'whoopingcough_14.wav', 'whoopin

In [13]:
wh = [{'0': ['whoopingcough_1.wav', 'whoopingcough_8.wav', 'whoopingcough_41.wav', 'unknown_403.wav', 'unknown_324.wav'], '1': ['whoopingcough_10.wav', 'whoopingcough_39.wav', 'whoopingcough_5.wav', 'unknown_583.wav', 'unknown_1186.wav'], '2': ['whoopingcough_11.wav', 'whoopingcough_45.wav', 'whoopingcough_50.wav', 'unknown_272.wav', 'unknown_823.wav'], '3': ['whoopingcough_12.wav', 'whoopingcough_48.wav', 'whoopingcough_26.wav', 'unknown_285.wav', 'unknown_311.wav'], '4': ['whoopingcough_13.wav', 'whoopingcough_52.wav', 'whoopingcough_33.wav', 'unknown_16.wav', 'unknown_1334.wav'], '5': ['whoopingcough_14.wav', 'whoopingcough_45.wav', 'whoopingcough_33.wav', 'unknown_784.wav', 'unknown_1081.wav'], '6': ['whoopingcough_15.wav', 'whoopingcough_48.wav', 'whoopingcough_40.wav', 'unknown_707.wav', 'unknown_478.wav'], '7': ['whoopingcough_16.wav', 'whoopingcough_57.wav', 'whoopingcough_54.wav', 'unknown_1489.wav', 'unknown_235.wav'], '8': ['whoopingcough_17.wav', 'whoopingcough_14.wav', 'whoopingcough_59.wav', 'unknown_1169.wav', 'unknown_873.wav'], '9': ['whoopingcough_18.wav', 'whoopingcough_35.wav', 'whoopingcough_60.wav', 'unknown_429.wav', 'unknown_842.wav'], '10': ['whoopingcough_19.wav', 'whoopingcough_23.wav', 'whoopingcough_16.wav', 'unknown_722.wav', 'unknown_296.wav'], '11': ['whoopingcough_2.wav', 'whoopingcough_54.wav', 'whoopingcough_19.wav', 'unknown_1252.wav', 'unknown_165.wav'], '12': ['whoopingcough_20.wav', 'whoopingcough_13.wav', 'whoopingcough_21.wav', 'unknown_246.wav', 'unknown_932.wav'], '13': ['whoopingcough_21.wav', 'whoopingcough_56.wav', 'whoopingcough_37.wav', 'unknown_44.wav', 'unknown_965.wav'], '14': ['whoopingcough_22.wav', 'whoopingcough_52.wav', 'whoopingcough_30.wav', 'unknown_131.wav', 'unknown_1041.wav'], '15': ['whoopingcough_23.wav', 'whoopingcough_59.wav', 'whoopingcough_19.wav', 'unknown_760.wav', 'unknown_837.wav'], '16': ['whoopingcough_24.wav', 'whoopingcough_34.wav', 'whoopingcough_33.wav', 'unknown_1054.wav', 'unknown_1454.wav'], '17': ['whoopingcough_25.wav', 'whoopingcough_37.wav', 'whoopingcough_31.wav', 'unknown_181.wav', 'unknown_794.wav'], '18': ['whoopingcough_26.wav', 'whoopingcough_50.wav', 'whoopingcough_16.wav', 'unknown_828.wav', 'unknown_943.wav'], '19': ['whoopingcough_27.wav', 'whoopingcough_12.wav', 'whoopingcough_58.wav', 'unknown_523.wav', 'unknown_172.wav'], '20': ['whoopingcough_28.wav', 'whoopingcough_49.wav', 'whoopingcough_45.wav', 'unknown_869.wav', 'unknown_1125.wav'], '21': ['whoopingcough_29.wav', 'whoopingcough_49.wav', 'whoopingcough_30.wav', 'unknown_790.wav', 'unknown_729.wav'], '22': ['whoopingcough_3.wav', 'whoopingcough_17.wav', 'whoopingcough_43.wav', 'unknown_123.wav', 'unknown_981.wav'], '23': ['whoopingcough_30.wav', 'whoopingcough_47.wav', 'whoopingcough_59.wav', 'unknown_575.wav', 'unknown_85.wav'], '24': ['whoopingcough_31.wav', 'whoopingcough_28.wav', 'whoopingcough_38.wav', 'unknown_380.wav', 'unknown_1156.wav'], '25': ['whoopingcough_32.wav', 'whoopingcough_48.wav', 'whoopingcough_49.wav', 'unknown_457.wav', 'unknown_94.wav'], '26': ['whoopingcough_33.wav', 'whoopingcough_53.wav', 'whoopingcough_38.wav', 'unknown_1174.wav', 'unknown_569.wav'], '27': ['whoopingcough_34.wav', 'whoopingcough_18.wav', 'whoopingcough_55.wav', 'unknown_799.wav', 'unknown_800.wav'], '28': ['whoopingcough_35.wav', 'whoopingcough_40.wav', 'whoopingcough_16.wav', 'unknown_305.wav', 'unknown_427.wav'], '29': ['whoopingcough_36.wav', 'whoopingcough_2.wav', 'whoopingcough_45.wav', 'unknown_825.wav', 'unknown_214.wav'], '30': ['whoopingcough_37.wav', 'whoopingcough_45.wav', 'whoopingcough_2.wav', 'unknown_579.wav', 'unknown_583.wav'], '31': ['whoopingcough_38.wav', 'whoopingcough_30.wav', 'whoopingcough_49.wav', 'unknown_326.wav', 'unknown_306.wav'], '32': ['whoopingcough_39.wav', 'whoopingcough_46.wav', 'whoopingcough_15.wav', 'unknown_1173.wav', 'unknown_1378.wav'], '33': ['whoopingcough_4.wav', 'whoopingcough_44.wav', 'whoopingcough_54.wav', 'unknown_189.wav', 'unknown_734.wav'], '34': ['whoopingcough_40.wav', 'whoopingcough_18.wav', 'whoopingcough_46.wav', 'unknown_274.wav', 'unknown_1063.wav'], '35': ['whoopingcough_41.wav', 'whoopingcough_1.wav', 'whoopingcough_53.wav', 'unknown_1260.wav', 'unknown_422.wav'], '36': ['whoopingcough_42.wav', 'whoopingcough_11.wav', 'whoopingcough_25.wav', 'unknown_441.wav', 'unknown_1534.wav'], '37': ['whoopingcough_43.wav', 'whoopingcough_55.wav', 'whoopingcough_33.wav', 'unknown_801.wav', 'unknown_1002.wav'], '38': ['whoopingcough_44.wav', 'whoopingcough_41.wav', 'whoopingcough_52.wav', 'unknown_537.wav', 'unknown_654.wav'], '39': ['whoopingcough_45.wav', 'whoopingcough_28.wav', 'whoopingcough_44.wav', 'unknown_564.wav', 'unknown_394.wav'], '40': ['whoopingcough_46.wav', 'whoopingcough_53.wav', 'whoopingcough_27.wav', 'unknown_313.wav', 'unknown_272.wav'], '41': ['whoopingcough_47.wav', 'whoopingcough_48.wav', 'whoopingcough_26.wav', 'unknown_294.wav', 'unknown_820.wav'], '42': ['whoopingcough_48.wav', 'whoopingcough_1.wav', 'whoopingcough_18.wav', 'unknown_1250.wav', 'unknown_1542.wav'], '43': ['whoopingcough_49.wav', 'whoopingcough_6.wav', 'whoopingcough_48.wav', 'unknown_588.wav', 'unknown_776.wav'], '44': ['whoopingcough_5.wav', 'whoopingcough_60.wav', 'whoopingcough_1.wav', 'unknown_269.wav', 'unknown_641.wav'], '45': ['whoopingcough_50.wav', 'whoopingcough_54.wav', 'whoopingcough_58.wav', 'unknown_1142.wav', 'unknown_1501.wav'], '46': ['whoopingcough_51.wav', 'whoopingcough_14.wav', 'whoopingcough_38.wav', 'unknown_691.wav', 'unknown_35.wav'], '47': ['whoopingcough_52.wav', 'whoopingcough_7.wav', 'whoopingcough_39.wav', 'unknown_915.wav', 'unknown_920.wav'], '48': ['whoopingcough_53.wav', 'whoopingcough_43.wav', 'whoopingcough_26.wav', 'unknown_1047.wav', 'unknown_1435.wav'], '49': ['whoopingcough_54.wav', 'whoopingcough_9.wav', 'whoopingcough_49.wav', 'unknown_1442.wav', 'unknown_1199.wav'], '50': ['whoopingcough_55.wav', 'whoopingcough_52.wav', 'whoopingcough_6.wav', 'unknown_71.wav', 'unknown_365.wav'], '51': ['whoopingcough_56.wav', 'whoopingcough_13.wav', 'whoopingcough_59.wav', 'unknown_1217.wav', 'unknown_1548.wav'], '52': ['whoopingcough_57.wav', 'whoopingcough_45.wav', 'whoopingcough_59.wav', 'unknown_961.wav', 'unknown_717.wav'], '53': ['whoopingcough_58.wav', 'whoopingcough_27.wav', 'whoopingcough_17.wav', 'unknown_1263.wav', 'unknown_153.wav'], '54': ['whoopingcough_59.wav', 'whoopingcough_50.wav', 'whoopingcough_13.wav', 'unknown_425.wav', 'unknown_647.wav'], '55': ['whoopingcough_6.wav', 'whoopingcough_33.wav', 'whoopingcough_27.wav', 'unknown_674.wav', 'unknown_1284.wav'], '56': ['whoopingcough_60.wav', 'whoopingcough_13.wav', 'whoopingcough_43.wav', 'unknown_682.wav', 'unknown_784.wav'], '57': ['whoopingcough_7.wav', 'whoopingcough_15.wav', 'whoopingcough_4.wav', 'unknown_117.wav', 'unknown_357.wav'], '58': ['whoopingcough_8.wav', 'whoopingcough_53.wav', 'whoopingcough_55.wav', 'unknown_1504.wav', 'unknown_997.wav'], '59': ['whoopingcough_9.wav', 'whoopingcough_46.wav', 'whoopingcough_55.wav', 'unknown_659.wav', 'unknown_524.wav']}, {'0': ['whoopingcough_1.wav', 'whoopingcough_58.wav', 'whoopingcough_20.wav', 'unknown_830.wav', 'unknown_889.wav'], '1': ['whoopingcough_10.wav', 'whoopingcough_60.wav', 'whoopingcough_42.wav', 'unknown_988.wav', 'unknown_665.wav'], '2': ['whoopingcough_11.wav', 'whoopingcough_15.wav', 'whoopingcough_46.wav', 'unknown_1173.wav', 'unknown_435.wav'], '3': ['whoopingcough_12.wav', 'whoopingcough_30.wav', 'whoopingcough_41.wav', 'unknown_647.wav', 'unknown_474.wav'], '4': ['whoopingcough_13.wav', 'whoopingcough_47.wav', 'whoopingcough_28.wav', 'unknown_1440.wav', 'unknown_1338.wav'], '5': ['whoopingcough_14.wav', 'whoopingcough_27.wav', 'whoopingcough_41.wav', 'unknown_567.wav', 'unknown_1455.wav'], '6': ['whoopingcough_15.wav', 'whoopingcough_1.wav', 'whoopingcough_18.wav', 'unknown_415.wav', 'unknown_39.wav'], '7': ['whoopingcough_16.wav', 'whoopingcough_57.wav', 'whoopingcough_9.wav', 'unknown_1391.wav', 'unknown_1551.wav'], '8': ['whoopingcough_17.wav', 'whoopingcough_39.wav', 'whoopingcough_22.wav', 'unknown_1207.wav', 'unknown_760.wav'], '9': ['whoopingcough_18.wav', 'whoopingcough_49.wav', 'whoopingcough_46.wav', 'unknown_1090.wav', 'unknown_1463.wav'], '10': ['whoopingcough_19.wav', 'whoopingcough_2.wav', 'whoopingcough_27.wav', 'unknown_130.wav', 'unknown_671.wav'], '11': ['whoopingcough_2.wav', 'whoopingcough_60.wav', 'whoopingcough_4.wav', 'unknown_1529.wav', 'unknown_295.wav'], '12': ['whoopingcough_20.wav', 'whoopingcough_23.wav', 'whoopingcough_38.wav', 'unknown_1066.wav', 'unknown_1365.wav'], '13': ['whoopingcough_21.wav', 'whoopingcough_58.wav', 'whoopingcough_59.wav', 'unknown_1384.wav', 'unknown_1416.wav'], '14': ['whoopingcough_22.wav', 'whoopingcough_26.wav', 'whoopingcough_44.wav', 'unknown_635.wav', 'unknown_867.wav'], '15': ['whoopingcough_23.wav', 'whoopingcough_49.wav', 'whoopingcough_6.wav', 'unknown_681.wav', 'unknown_129.wav'], '16': ['whoopingcough_24.wav', 'whoopingcough_42.wav', 'whoopingcough_34.wav', 'unknown_1220.wav', 'unknown_1239.wav'], '17': ['whoopingcough_25.wav', 'whoopingcough_1.wav', 'whoopingcough_42.wav', 'unknown_687.wav', 'unknown_491.wav'], '18': ['whoopingcough_26.wav', 'whoopingcough_37.wav', 'whoopingcough_27.wav', 'unknown_482.wav', 'unknown_1087.wav'], '19': ['whoopingcough_27.wav', 'whoopingcough_22.wav', 'whoopingcough_37.wav', 'unknown_688.wav', 'unknown_1145.wav'], '20': ['whoopingcough_28.wav', 'whoopingcough_50.wav', 'whoopingcough_58.wav', 'unknown_820.wav', 'unknown_167.wav'], '21': ['whoopingcough_29.wav', 'whoopingcough_36.wav', 'whoopingcough_26.wav', 'unknown_1325.wav', 'unknown_1457.wav'], '22': ['whoopingcough_3.wav', 'whoopingcough_48.wav', 'whoopingcough_10.wav', 'unknown_1354.wav', 'unknown_922.wav'], '23': ['whoopingcough_30.wav', 'whoopingcough_36.wav', 'whoopingcough_3.wav', 'unknown_297.wav', 'unknown_760.wav'], '24': ['whoopingcough_31.wav', 'whoopingcough_53.wav', 'whoopingcough_13.wav', 'unknown_57.wav', 'unknown_998.wav'], '25': ['whoopingcough_32.wav', 'whoopingcough_55.wav', 'whoopingcough_59.wav', 'unknown_615.wav', 'unknown_123.wav'], '26': ['whoopingcough_33.wav', 'whoopingcough_51.wav', 'whoopingcough_4.wav', 'unknown_505.wav', 'unknown_1057.wav'], '27': ['whoopingcough_34.wav', 'whoopingcough_27.wav', 'whoopingcough_44.wav', 'unknown_1547.wav', 'unknown_1410.wav'], '28': ['whoopingcough_35.wav', 'whoopingcough_50.wav', 'whoopingcough_33.wav', 'unknown_669.wav', 'unknown_863.wav'], '29': ['whoopingcough_36.wav', 'whoopingcough_18.wav', 'whoopingcough_33.wav', 'unknown_350.wav', 'unknown_99.wav'], '30': ['whoopingcough_37.wav', 'whoopingcough_16.wav', 'whoopingcough_18.wav', 'unknown_1380.wav', 'unknown_1547.wav'], '31': ['whoopingcough_38.wav', 'whoopingcough_5.wav', 'whoopingcough_47.wav', 'unknown_618.wav', 'unknown_542.wav'], '32': ['whoopingcough_39.wav', 'whoopingcough_42.wav', 'whoopingcough_4.wav', 'unknown_904.wav', 'unknown_1213.wav'], '33': ['whoopingcough_4.wav', 'whoopingcough_5.wav', 'whoopingcough_38.wav', 'unknown_1187.wav', 'unknown_562.wav'], '34': ['whoopingcough_40.wav', 'whoopingcough_34.wav', 'whoopingcough_38.wav', 'unknown_1507.wav', 'unknown_1475.wav'], '35': ['whoopingcough_41.wav', 'whoopingcough_5.wav', 'whoopingcough_34.wav', 'unknown_1401.wav', 'unknown_106.wav'], '36': ['whoopingcough_42.wav', 'whoopingcough_14.wav', 'whoopingcough_16.wav', 'unknown_826.wav', 'unknown_1095.wav'], '37': ['whoopingcough_43.wav', 'whoopingcough_58.wav', 'whoopingcough_55.wav', 'unknown_1122.wav', 'unknown_826.wav'], '38': ['whoopingcough_44.wav', 'whoopingcough_51.wav', 'whoopingcough_54.wav', 'unknown_66.wav', 'unknown_85.wav'], '39': ['whoopingcough_45.wav', 'whoopingcough_11.wav', 'whoopingcough_4.wav', 'unknown_298.wav', 'unknown_891.wav'], '40': ['whoopingcough_46.wav', 'whoopingcough_55.wav', 'whoopingcough_8.wav', 'unknown_665.wav', 'unknown_323.wav'], '41': ['whoopingcough_47.wav', 'whoopingcough_7.wav', 'whoopingcough_59.wav', 'unknown_547.wav', 'unknown_868.wav'], '42': ['whoopingcough_48.wav', 'whoopingcough_8.wav', 'whoopingcough_38.wav', 'unknown_384.wav', 'unknown_101.wav'], '43': ['whoopingcough_49.wav', 'whoopingcough_44.wav', 'whoopingcough_48.wav', 'unknown_875.wav', 'unknown_376.wav'], '44': ['whoopingcough_5.wav', 'whoopingcough_59.wav', 'whoopingcough_14.wav', 'unknown_1400.wav', 'unknown_966.wav'], '45': ['whoopingcough_50.wav', 'whoopingcough_7.wav', 'whoopingcough_1.wav', 'unknown_1018.wav', 'unknown_1500.wav'], '46': ['whoopingcough_51.wav', 'whoopingcough_54.wav', 'whoopingcough_56.wav', 'unknown_1512.wav', 'unknown_906.wav'], '47': ['whoopingcough_52.wav', 'whoopingcough_17.wav', 'whoopingcough_16.wav', 'unknown_710.wav', 'unknown_553.wav'], '48': ['whoopingcough_53.wav', 'whoopingcough_54.wav', 'whoopingcough_56.wav', 'unknown_300.wav', 'unknown_46.wav'], '49': ['whoopingcough_54.wav', 'whoopingcough_46.wav', 'whoopingcough_38.wav', 'unknown_891.wav', 'unknown_912.wav'], '50': ['whoopingcough_55.wav', 'whoopingcough_28.wav', 'whoopingcough_4.wav', 'unknown_1123.wav', 'unknown_864.wav'], '51': ['whoopingcough_56.wav', 'whoopingcough_48.wav', 'whoopingcough_41.wav', 'unknown_728.wav', 'unknown_563.wav'], '52': ['whoopingcough_57.wav', 'whoopingcough_15.wav', 'whoopingcough_11.wav', 'unknown_1382.wav', 'unknown_1500.wav'], '53': ['whoopingcough_58.wav', 'whoopingcough_28.wav', 'whoopingcough_39.wav', 'unknown_268.wav', 'unknown_554.wav'], '54': ['whoopingcough_59.wav', 'whoopingcough_32.wav', 'whoopingcough_27.wav', 'unknown_457.wav', 'unknown_4.wav'], '55': ['whoopingcough_6.wav', 'whoopingcough_21.wav', 'whoopingcough_42.wav', 'unknown_1069.wav', 'unknown_662.wav'], '56': ['whoopingcough_60.wav', 'whoopingcough_52.wav', 'whoopingcough_9.wav', 'unknown_1469.wav', 'unknown_1060.wav'], '57': ['whoopingcough_7.wav', 'whoopingcough_13.wav', 'whoopingcough_33.wav', 'unknown_845.wav', 'unknown_413.wav'], '58': ['whoopingcough_8.wav', 'whoopingcough_45.wav', 'whoopingcough_40.wav', 'unknown_649.wav', 'unknown_755.wav'], '59': ['whoopingcough_9.wav', 'whoopingcough_5.wav', 'whoopingcough_44.wav', 'unknown_177.wav', 'unknown_391.wav']}, {'0': ['whoopingcough_1.wav', 'whoopingcough_10.wav', 'whoopingcough_24.wav', 'unknown_531.wav', 'unknown_942.wav'], '1': ['whoopingcough_10.wav', 'whoopingcough_19.wav', 'whoopingcough_60.wav', 'unknown_794.wav', 'unknown_469.wav'], '2': ['whoopingcough_11.wav', 'whoopingcough_55.wav', 'whoopingcough_36.wav', 'unknown_839.wav', 'unknown_197.wav'], '3': ['whoopingcough_12.wav', 'whoopingcough_33.wav', 'whoopingcough_25.wav', 'unknown_495.wav', 'unknown_1425.wav'], '4': ['whoopingcough_13.wav', 'whoopingcough_31.wav', 'whoopingcough_32.wav', 'unknown_1014.wav', 'unknown_63.wav'], '5': ['whoopingcough_14.wav', 'whoopingcough_15.wav', 'whoopingcough_31.wav', 'unknown_1090.wav', 'unknown_840.wav'], '6': ['whoopingcough_15.wav', 'whoopingcough_32.wav', 'whoopingcough_55.wav', 'unknown_696.wav', 'unknown_452.wav'], '7': ['whoopingcough_16.wav', 'whoopingcough_44.wav', 'whoopingcough_48.wav', 'unknown_1298.wav', 'unknown_552.wav'], '8': ['whoopingcough_17.wav', 'whoopingcough_8.wav', 'whoopingcough_29.wav', 'unknown_717.wav', 'unknown_720.wav'], '9': ['whoopingcough_18.wav', 'whoopingcough_26.wav', 'whoopingcough_53.wav', 'unknown_1013.wav', 'unknown_1037.wav'], '10': ['whoopingcough_19.wav', 'whoopingcough_7.wav', 'whoopingcough_33.wav', 'unknown_144.wav', 'unknown_227.wav'], '11': ['whoopingcough_2.wav', 'whoopingcough_52.wav', 'whoopingcough_50.wav', 'unknown_1374.wav', 'unknown_672.wav'], '12': ['whoopingcough_20.wav', 'whoopingcough_4.wav', 'whoopingcough_40.wav', 'unknown_224.wav', 'unknown_734.wav'], '13': ['whoopingcough_21.wav', 'whoopingcough_38.wav', 'whoopingcough_44.wav', 'unknown_1389.wav', 'unknown_1415.wav'], '14': ['whoopingcough_22.wav', 'whoopingcough_10.wav', 'whoopingcough_25.wav', 'unknown_186.wav', 'unknown_1075.wav'], '15': ['whoopingcough_23.wav', 'whoopingcough_2.wav', 'whoopingcough_22.wav', 'unknown_331.wav', 'unknown_919.wav'], '16': ['whoopingcough_24.wav', 'whoopingcough_59.wav', 'whoopingcough_7.wav', 'unknown_1054.wav', 'unknown_231.wav'], '17': ['whoopingcough_25.wav', 'whoopingcough_19.wav', 'whoopingcough_26.wav', 'unknown_1314.wav', 'unknown_241.wav'], '18': ['whoopingcough_26.wav', 'whoopingcough_49.wav', 'whoopingcough_24.wav', 'unknown_734.wav', 'unknown_1137.wav'], '19': ['whoopingcough_27.wav', 'whoopingcough_24.wav', 'whoopingcough_15.wav', 'unknown_364.wav', 'unknown_488.wav'], '20': ['whoopingcough_28.wav', 'whoopingcough_4.wav', 'whoopingcough_55.wav', 'unknown_642.wav', 'unknown_46.wav'], '21': ['whoopingcough_29.wav', 'whoopingcough_30.wav', 'whoopingcough_28.wav', 'unknown_1498.wav', 'unknown_1546.wav'], '22': ['whoopingcough_3.wav', 'whoopingcough_24.wav', 'whoopingcough_18.wav', 'unknown_1537.wav', 'unknown_715.wav'], '23': ['whoopingcough_30.wav', 'whoopingcough_56.wav', 'whoopingcough_52.wav', 'unknown_769.wav', 'unknown_1164.wav'], '24': ['whoopingcough_31.wav', 'whoopingcough_17.wav', 'whoopingcough_49.wav', 'unknown_1351.wav', 'unknown_663.wav'], '25': ['whoopingcough_32.wav', 'whoopingcough_27.wav', 'whoopingcough_45.wav', 'unknown_1351.wav', 'unknown_1401.wav'], '26': ['whoopingcough_33.wav', 'whoopingcough_20.wav', 'whoopingcough_25.wav', 'unknown_215.wav', 'unknown_132.wav'], '27': ['whoopingcough_34.wav', 'whoopingcough_38.wav', 'whoopingcough_16.wav', 'unknown_1236.wav', 'unknown_778.wav'], '28': ['whoopingcough_35.wav', 'whoopingcough_28.wav', 'whoopingcough_21.wav', 'unknown_248.wav', 'unknown_717.wav'], '29': ['whoopingcough_36.wav', 'whoopingcough_43.wav', 'whoopingcough_31.wav', 'unknown_1422.wav', 'unknown_1148.wav'], '30': ['whoopingcough_37.wav', 'whoopingcough_38.wav', 'whoopingcough_19.wav', 'unknown_165.wav', 'unknown_166.wav'], '31': ['whoopingcough_38.wav', 'whoopingcough_2.wav', 'whoopingcough_50.wav', 'unknown_660.wav', 'unknown_1184.wav'], '32': ['whoopingcough_39.wav', 'whoopingcough_53.wav', 'whoopingcough_17.wav', 'unknown_1241.wav', 'unknown_1475.wav'], '33': ['whoopingcough_4.wav', 'whoopingcough_32.wav', 'whoopingcough_46.wav', 'unknown_828.wav', 'unknown_907.wav'], '34': ['whoopingcough_40.wav', 'whoopingcough_16.wav', 'whoopingcough_36.wav', 'unknown_67.wav', 'unknown_8.wav'], '35': ['whoopingcough_41.wav', 'whoopingcough_59.wav', 'whoopingcough_3.wav', 'unknown_670.wav', 'unknown_1177.wav'], '36': ['whoopingcough_42.wav', 'whoopingcough_11.wav', 'whoopingcough_10.wav', 'unknown_4.wav', 'unknown_854.wav'], '37': ['whoopingcough_43.wav', 'whoopingcough_51.wav', 'whoopingcough_59.wav', 'unknown_1035.wav', 'unknown_809.wav'], '38': ['whoopingcough_44.wav', 'whoopingcough_16.wav', 'whoopingcough_37.wav', 'unknown_64.wav', 'unknown_324.wav'], '39': ['whoopingcough_45.wav', 'whoopingcough_3.wav', 'whoopingcough_17.wav', 'unknown_1377.wav', 'unknown_990.wav'], '40': ['whoopingcough_46.wav', 'whoopingcough_1.wav', 'whoopingcough_26.wav', 'unknown_1466.wav', 'unknown_763.wav'], '41': ['whoopingcough_47.wav', 'whoopingcough_7.wav', 'whoopingcough_59.wav', 'unknown_51.wav', 'unknown_1285.wav'], '42': ['whoopingcough_48.wav', 'whoopingcough_11.wav', 'whoopingcough_16.wav', 'unknown_1086.wav', 'unknown_189.wav'], '43': ['whoopingcough_49.wav', 'whoopingcough_53.wav', 'whoopingcough_31.wav', 'unknown_229.wav', 'unknown_1077.wav'], '44': ['whoopingcough_5.wav', 'whoopingcough_15.wav', 'whoopingcough_39.wav', 'unknown_308.wav', 'unknown_1355.wav'], '45': ['whoopingcough_50.wav', 'whoopingcough_55.wav', 'whoopingcough_15.wav', 'unknown_481.wav', 'unknown_108.wav'], '46': ['whoopingcough_51.wav', 'whoopingcough_59.wav', 'whoopingcough_15.wav', 'unknown_1262.wav', 'unknown_1301.wav'], '47': ['whoopingcough_52.wav', 'whoopingcough_55.wav', 'whoopingcough_29.wav', 'unknown_459.wav', 'unknown_1000.wav'], '48': ['whoopingcough_53.wav', 'whoopingcough_20.wav', 'whoopingcough_21.wav', 'unknown_1458.wav', 'unknown_825.wav'], '49': ['whoopingcough_54.wav', 'whoopingcough_26.wav', 'whoopingcough_31.wav', 'unknown_1164.wav', 'unknown_1348.wav'], '50': ['whoopingcough_55.wav', 'whoopingcough_41.wav', 'whoopingcough_25.wav', 'unknown_667.wav', 'unknown_113.wav'], '51': ['whoopingcough_56.wav', 'whoopingcough_45.wav', 'whoopingcough_21.wav', 'unknown_301.wav', 'unknown_841.wav'], '52': ['whoopingcough_57.wav', 'whoopingcough_29.wav', 'whoopingcough_31.wav', 'unknown_477.wav', 'unknown_1323.wav'], '53': ['whoopingcough_58.wav', 'whoopingcough_6.wav', 'whoopingcough_14.wav', 'unknown_854.wav', 'unknown_590.wav'], '54': ['whoopingcough_59.wav', 'whoopingcough_58.wav', 'whoopingcough_7.wav', 'unknown_503.wav', 'unknown_268.wav'], '55': ['whoopingcough_6.wav', 'whoopingcough_12.wav', 'whoopingcough_8.wav', 'unknown_51.wav', 'unknown_693.wav'], '56': ['whoopingcough_60.wav', 'whoopingcough_30.wav', 'whoopingcough_2.wav', 'unknown_978.wav', 'unknown_183.wav'], '57': ['whoopingcough_7.wav', 'whoopingcough_19.wav', 'whoopingcough_27.wav', 'unknown_16.wav', 'unknown_903.wav'], '58': ['whoopingcough_8.wav', 'whoopingcough_31.wav', 'whoopingcough_13.wav', 'unknown_1370.wav', 'unknown_1208.wav'], '59': ['whoopingcough_9.wav', 'whoopingcough_15.wav', 'whoopingcough_49.wav', 'unknown_1505.wav', 'unknown_94.wav']}, {'0': ['whoopingcough_1.wav', 'whoopingcough_31.wav', 'whoopingcough_60.wav', 'unknown_947.wav', 'unknown_146.wav'], '1': ['whoopingcough_10.wav', 'whoopingcough_19.wav', 'whoopingcough_34.wav', 'unknown_764.wav', 'unknown_228.wav'], '2': ['whoopingcough_11.wav', 'whoopingcough_42.wav', 'whoopingcough_17.wav', 'unknown_510.wav', 'unknown_875.wav'], '3': ['whoopingcough_12.wav', 'whoopingcough_13.wav', 'whoopingcough_21.wav', 'unknown_1208.wav', 'unknown_217.wav'], '4': ['whoopingcough_13.wav', 'whoopingcough_38.wav', 'whoopingcough_55.wav', 'unknown_92.wav', 'unknown_1407.wav'], '5': ['whoopingcough_14.wav', 'whoopingcough_56.wav', 'whoopingcough_3.wav', 'unknown_584.wav', 'unknown_334.wav'], '6': ['whoopingcough_15.wav', 'whoopingcough_46.wav', 'whoopingcough_33.wav', 'unknown_594.wav', 'unknown_828.wav'], '7': ['whoopingcough_16.wav', 'whoopingcough_39.wav', 'whoopingcough_26.wav', 'unknown_1038.wav', 'unknown_460.wav'], '8': ['whoopingcough_17.wav', 'whoopingcough_16.wav', 'whoopingcough_31.wav', 'unknown_1321.wav', 'unknown_907.wav'], '9': ['whoopingcough_18.wav', 'whoopingcough_10.wav', 'whoopingcough_47.wav', 'unknown_1338.wav', 'unknown_1313.wav'], '10': ['whoopingcough_19.wav', 'whoopingcough_9.wav', 'whoopingcough_37.wav', 'unknown_780.wav', 'unknown_367.wav'], '11': ['whoopingcough_2.wav', 'whoopingcough_35.wav', 'whoopingcough_31.wav', 'unknown_1029.wav', 'unknown_838.wav'], '12': ['whoopingcough_20.wav', 'whoopingcough_6.wav', 'whoopingcough_5.wav', 'unknown_736.wav', 'unknown_827.wav'], '13': ['whoopingcough_21.wav', 'whoopingcough_48.wav', 'whoopingcough_46.wav', 'unknown_414.wav', 'unknown_712.wav'], '14': ['whoopingcough_22.wav', 'whoopingcough_11.wav', 'whoopingcough_47.wav', 'unknown_1009.wav', 'unknown_641.wav'], '15': ['whoopingcough_23.wav', 'whoopingcough_54.wav', 'whoopingcough_13.wav', 'unknown_897.wav', 'unknown_1243.wav'], '16': ['whoopingcough_24.wav', 'whoopingcough_19.wav', 'whoopingcough_31.wav', 'unknown_51.wav', 'unknown_132.wav'], '17': ['whoopingcough_25.wav', 'whoopingcough_58.wav', 'whoopingcough_39.wav', 'unknown_1000.wav', 'unknown_359.wav'], '18': ['whoopingcough_26.wav', 'whoopingcough_22.wav', 'whoopingcough_30.wav', 'unknown_374.wav', 'unknown_1531.wav'], '19': ['whoopingcough_27.wav', 'whoopingcough_47.wav', 'whoopingcough_1.wav', 'unknown_921.wav', 'unknown_575.wav'], '20': ['whoopingcough_28.wav', 'whoopingcough_50.wav', 'whoopingcough_58.wav', 'unknown_1510.wav', 'unknown_730.wav'], '21': ['whoopingcough_29.wav', 'whoopingcough_24.wav', 'whoopingcough_14.wav', 'unknown_1213.wav', 'unknown_805.wav'], '22': ['whoopingcough_3.wav', 'whoopingcough_41.wav', 'whoopingcough_38.wav', 'unknown_535.wav', 'unknown_1247.wav'], '23': ['whoopingcough_30.wav', 'whoopingcough_51.wav', 'whoopingcough_54.wav', 'unknown_1310.wav', 'unknown_967.wav'], '24': ['whoopingcough_31.wav', 'whoopingcough_60.wav', 'whoopingcough_58.wav', 'unknown_374.wav', 'unknown_1052.wav'], '25': ['whoopingcough_32.wav', 'whoopingcough_19.wav', 'whoopingcough_5.wav', 'unknown_599.wav', 'unknown_558.wav'], '26': ['whoopingcough_33.wav', 'whoopingcough_55.wav', 'whoopingcough_24.wav', 'unknown_1313.wav', 'unknown_49.wav'], '27': ['whoopingcough_34.wav', 'whoopingcough_6.wav', 'whoopingcough_38.wav', 'unknown_138.wav', 'unknown_748.wav'], '28': ['whoopingcough_35.wav', 'whoopingcough_45.wav', 'whoopingcough_14.wav', 'unknown_1056.wav', 'unknown_7.wav'], '29': ['whoopingcough_36.wav', 'whoopingcough_2.wav', 'whoopingcough_9.wav', 'unknown_688.wav', 'unknown_1011.wav'], '30': ['whoopingcough_37.wav', 'whoopingcough_9.wav', 'whoopingcough_7.wav', 'unknown_748.wav', 'unknown_582.wav'], '31': ['whoopingcough_38.wav', 'whoopingcough_23.wav', 'whoopingcough_43.wav', 'unknown_1157.wav', 'unknown_423.wav'], '32': ['whoopingcough_39.wav', 'whoopingcough_7.wav', 'whoopingcough_56.wav', 'unknown_416.wav', 'unknown_120.wav'], '33': ['whoopingcough_4.wav', 'whoopingcough_39.wav', 'whoopingcough_19.wav', 'unknown_582.wav', 'unknown_646.wav'], '34': ['whoopingcough_40.wav', 'whoopingcough_38.wav', 'whoopingcough_54.wav', 'unknown_812.wav', 'unknown_903.wav'], '35': ['whoopingcough_41.wav', 'whoopingcough_13.wav', 'whoopingcough_36.wav', 'unknown_126.wav', 'unknown_1173.wav'], '36': ['whoopingcough_42.wav', 'whoopingcough_57.wav', 'whoopingcough_40.wav', 'unknown_1485.wav', 'unknown_1510.wav'], '37': ['whoopingcough_43.wav', 'whoopingcough_6.wav', 'whoopingcough_42.wav', 'unknown_432.wav', 'unknown_112.wav'], '38': ['whoopingcough_44.wav', 'whoopingcough_56.wav', 'whoopingcough_26.wav', 'unknown_200.wav', 'unknown_718.wav'], '39': ['whoopingcough_45.wav', 'whoopingcough_18.wav', 'whoopingcough_46.wav', 'unknown_1429.wav', 'unknown_119.wav'], '40': ['whoopingcough_46.wav', 'whoopingcough_12.wav', 'whoopingcough_11.wav', 'unknown_361.wav', 'unknown_1542.wav'], '41': ['whoopingcough_47.wav', 'whoopingcough_27.wav', 'whoopingcough_2.wav', 'unknown_1241.wav', 'unknown_1525.wav'], '42': ['whoopingcough_48.wav', 'whoopingcough_8.wav', 'whoopingcough_19.wav', 'unknown_613.wav', 'unknown_278.wav'], '43': ['whoopingcough_49.wav', 'whoopingcough_42.wav', 'whoopingcough_58.wav', 'unknown_1494.wav', 'unknown_226.wav'], '44': ['whoopingcough_5.wav', 'whoopingcough_7.wav', 'whoopingcough_4.wav', 'unknown_1438.wav', 'unknown_426.wav'], '45': ['whoopingcough_50.wav', 'whoopingcough_26.wav', 'whoopingcough_34.wav', 'unknown_331.wav', 'unknown_56.wav'], '46': ['whoopingcough_51.wav', 'whoopingcough_50.wav', 'whoopingcough_37.wav', 'unknown_1488.wav', 'unknown_886.wav'], '47': ['whoopingcough_52.wav', 'whoopingcough_36.wav', 'whoopingcough_48.wav', 'unknown_30.wav', 'unknown_1542.wav'], '48': ['whoopingcough_53.wav', 'whoopingcough_31.wav', 'whoopingcough_42.wav', 'unknown_1458.wav', 'unknown_321.wav'], '49': ['whoopingcough_54.wav', 'whoopingcough_45.wav', 'whoopingcough_10.wav', 'unknown_1276.wav', 'unknown_184.wav'], '50': ['whoopingcough_55.wav', 'whoopingcough_25.wav', 'whoopingcough_22.wav', 'unknown_1083.wav', 'unknown_1169.wav'], '51': ['whoopingcough_56.wav', 'whoopingcough_13.wav', 'whoopingcough_4.wav', 'unknown_986.wav', 'unknown_1549.wav'], '52': ['whoopingcough_57.wav', 'whoopingcough_33.wav', 'whoopingcough_9.wav', 'unknown_1061.wav', 'unknown_1199.wav'], '53': ['whoopingcough_58.wav', 'whoopingcough_13.wav', 'whoopingcough_2.wav', 'unknown_1095.wav', 'unknown_253.wav'], '54': ['whoopingcough_59.wav', 'whoopingcough_4.wav', 'whoopingcough_43.wav', 'unknown_932.wav', 'unknown_900.wav'], '55': ['whoopingcough_6.wav', 'whoopingcough_3.wav', 'whoopingcough_43.wav', 'unknown_1172.wav', 'unknown_1264.wav'], '56': ['whoopingcough_60.wav', 'whoopingcough_5.wav', 'whoopingcough_4.wav', 'unknown_674.wav', 'unknown_912.wav'], '57': ['whoopingcough_7.wav', 'whoopingcough_48.wav', 'whoopingcough_12.wav', 'unknown_824.wav', 'unknown_1256.wav'], '58': ['whoopingcough_8.wav', 'whoopingcough_22.wav', 'whoopingcough_4.wav', 'unknown_762.wav', 'unknown_372.wav'], '59': ['whoopingcough_9.wav', 'whoopingcough_36.wav', 'whoopingcough_37.wav', 'unknown_501.wav', 'unknown_1212.wav']}, {'0': ['whoopingcough_1.wav', 'whoopingcough_30.wav', 'whoopingcough_16.wav', 'unknown_1069.wav', 'unknown_169.wav'], '1': ['whoopingcough_10.wav', 'whoopingcough_42.wav', 'whoopingcough_31.wav', 'unknown_1535.wav', 'unknown_716.wav'], '2': ['whoopingcough_11.wav', 'whoopingcough_30.wav', 'whoopingcough_9.wav', 'unknown_1098.wav', 'unknown_1010.wav'], '3': ['whoopingcough_12.wav', 'whoopingcough_29.wav', 'whoopingcough_19.wav', 'unknown_775.wav', 'unknown_1369.wav'], '4': ['whoopingcough_13.wav', 'whoopingcough_14.wav', 'whoopingcough_25.wav', 'unknown_1493.wav', 'unknown_514.wav'], '5': ['whoopingcough_14.wav', 'whoopingcough_8.wav', 'whoopingcough_10.wav', 'unknown_176.wav', 'unknown_889.wav'], '6': ['whoopingcough_15.wav', 'whoopingcough_53.wav', 'whoopingcough_4.wav', 'unknown_480.wav', 'unknown_729.wav'], '7': ['whoopingcough_16.wav', 'whoopingcough_12.wav', 'whoopingcough_38.wav', 'unknown_45.wav', 'unknown_88.wav'], '8': ['whoopingcough_17.wav', 'whoopingcough_31.wav', 'whoopingcough_33.wav', 'unknown_616.wav', 'unknown_1279.wav'], '9': ['whoopingcough_18.wav', 'whoopingcough_44.wav', 'whoopingcough_14.wav', 'unknown_274.wav', 'unknown_653.wav'], '10': ['whoopingcough_19.wav', 'whoopingcough_33.wav', 'whoopingcough_18.wav', 'unknown_79.wav', 'unknown_529.wav'], '11': ['whoopingcough_2.wav', 'whoopingcough_35.wav', 'whoopingcough_6.wav', 'unknown_853.wav', 'unknown_1010.wav'], '12': ['whoopingcough_20.wav', 'whoopingcough_46.wav', 'whoopingcough_2.wav', 'unknown_985.wav', 'unknown_17.wav'], '13': ['whoopingcough_21.wav', 'whoopingcough_15.wav', 'whoopingcough_8.wav', 'unknown_293.wav', 'unknown_102.wav'], '14': ['whoopingcough_22.wav', 'whoopingcough_54.wav', 'whoopingcough_16.wav', 'unknown_1325.wav', 'unknown_1086.wav'], '15': ['whoopingcough_23.wav', 'whoopingcough_54.wav', 'whoopingcough_16.wav', 'unknown_1065.wav', 'unknown_396.wav'], '16': ['whoopingcough_24.wav', 'whoopingcough_25.wav', 'whoopingcough_21.wav', 'unknown_426.wav', 'unknown_642.wav'], '17': ['whoopingcough_25.wav', 'whoopingcough_22.wav', 'whoopingcough_1.wav', 'unknown_622.wav', 'unknown_1026.wav'], '18': ['whoopingcough_26.wav', 'whoopingcough_16.wav', 'whoopingcough_8.wav', 'unknown_278.wav', 'unknown_42.wav'], '19': ['whoopingcough_27.wav', 'whoopingcough_19.wav', 'whoopingcough_9.wav', 'unknown_1499.wav', 'unknown_41.wav'], '20': ['whoopingcough_28.wav', 'whoopingcough_21.wav', 'whoopingcough_49.wav', 'unknown_866.wav', 'unknown_206.wav'], '21': ['whoopingcough_29.wav', 'whoopingcough_40.wav', 'whoopingcough_60.wav', 'unknown_62.wav', 'unknown_1118.wav'], '22': ['whoopingcough_3.wav', 'whoopingcough_25.wav', 'whoopingcough_46.wav', 'unknown_126.wav', 'unknown_69.wav'], '23': ['whoopingcough_30.wav', 'whoopingcough_26.wav', 'whoopingcough_2.wav', 'unknown_467.wav', 'unknown_183.wav'], '24': ['whoopingcough_31.wav', 'whoopingcough_39.wav', 'whoopingcough_38.wav', 'unknown_1294.wav', 'unknown_1456.wav'], '25': ['whoopingcough_32.wav', 'whoopingcough_50.wav', 'whoopingcough_27.wav', 'unknown_736.wav', 'unknown_293.wav'], '26': ['whoopingcough_33.wav', 'whoopingcough_9.wav', 'whoopingcough_46.wav', 'unknown_611.wav', 'unknown_511.wav'], '27': ['whoopingcough_34.wav', 'whoopingcough_52.wav', 'whoopingcough_9.wav', 'unknown_30.wav', 'unknown_805.wav'], '28': ['whoopingcough_35.wav', 'whoopingcough_47.wav', 'whoopingcough_48.wav', 'unknown_1381.wav', 'unknown_9.wav'], '29': ['whoopingcough_36.wav', 'whoopingcough_57.wav', 'whoopingcough_17.wav', 'unknown_1070.wav', 'unknown_1107.wav'], '30': ['whoopingcough_37.wav', 'whoopingcough_15.wav', 'whoopingcough_41.wav', 'unknown_173.wav', 'unknown_1230.wav'], '31': ['whoopingcough_38.wav', 'whoopingcough_31.wav', 'whoopingcough_15.wav', 'unknown_391.wav', 'unknown_202.wav'], '32': ['whoopingcough_39.wav', 'whoopingcough_1.wav', 'whoopingcough_18.wav', 'unknown_1289.wav', 'unknown_201.wav'], '33': ['whoopingcough_4.wav', 'whoopingcough_30.wav', 'whoopingcough_53.wav', 'unknown_731.wav', 'unknown_1439.wav'], '34': ['whoopingcough_40.wav', 'whoopingcough_24.wav', 'whoopingcough_11.wav', 'unknown_1042.wav', 'unknown_72.wav'], '35': ['whoopingcough_41.wav', 'whoopingcough_29.wav', 'whoopingcough_16.wav', 'unknown_751.wav', 'unknown_521.wav'], '36': ['whoopingcough_42.wav', 'whoopingcough_2.wav', 'whoopingcough_46.wav', 'unknown_167.wav', 'unknown_182.wav'], '37': ['whoopingcough_43.wav', 'whoopingcough_6.wav', 'whoopingcough_3.wav', 'unknown_315.wav', 'unknown_273.wav'], '38': ['whoopingcough_44.wav', 'whoopingcough_59.wav', 'whoopingcough_10.wav', 'unknown_113.wav', 'unknown_949.wav'], '39': ['whoopingcough_45.wav', 'whoopingcough_21.wav', 'whoopingcough_49.wav', 'unknown_1371.wav', 'unknown_136.wav'], '40': ['whoopingcough_46.wav', 'whoopingcough_10.wav', 'whoopingcough_17.wav', 'unknown_705.wav', 'unknown_339.wav'], '41': ['whoopingcough_47.wav', 'whoopingcough_30.wav', 'whoopingcough_8.wav', 'unknown_1079.wav', 'unknown_321.wav'], '42': ['whoopingcough_48.wav', 'whoopingcough_21.wav', 'whoopingcough_36.wav', 'unknown_1303.wav', 'unknown_199.wav'], '43': ['whoopingcough_49.wav', 'whoopingcough_22.wav', 'whoopingcough_26.wav', 'unknown_788.wav', 'unknown_701.wav'], '44': ['whoopingcough_5.wav', 'whoopingcough_37.wav', 'whoopingcough_31.wav', 'unknown_866.wav', 'unknown_874.wav'], '45': ['whoopingcough_50.wav', 'whoopingcough_46.wav', 'whoopingcough_5.wav', 'unknown_1490.wav', 'unknown_560.wav'], '46': ['whoopingcough_51.wav', 'whoopingcough_53.wav', 'whoopingcough_40.wav', 'unknown_1238.wav', 'unknown_866.wav'], '47': ['whoopingcough_52.wav', 'whoopingcough_55.wav', 'whoopingcough_59.wav', 'unknown_1231.wav', 'unknown_827.wav'], '48': ['whoopingcough_53.wav', 'whoopingcough_31.wav', 'whoopingcough_7.wav', 'unknown_886.wav', 'unknown_221.wav'], '49': ['whoopingcough_54.wav', 'whoopingcough_27.wav', 'whoopingcough_25.wav', 'unknown_228.wav', 'unknown_716.wav'], '50': ['whoopingcough_55.wav', 'whoopingcough_28.wav', 'whoopingcough_30.wav', 'unknown_375.wav', 'unknown_386.wav'], '51': ['whoopingcough_56.wav', 'whoopingcough_11.wav', 'whoopingcough_22.wav', 'unknown_992.wav', 'unknown_1470.wav'], '52': ['whoopingcough_57.wav', 'whoopingcough_50.wav', 'whoopingcough_8.wav', 'unknown_1314.wav', 'unknown_1498.wav'], '53': ['whoopingcough_58.wav', 'whoopingcough_55.wav', 'whoopingcough_2.wav', 'unknown_1240.wav', 'unknown_454.wav'], '54': ['whoopingcough_59.wav', 'whoopingcough_44.wav', 'whoopingcough_23.wav', 'unknown_242.wav', 'unknown_639.wav'], '55': ['whoopingcough_6.wav', 'whoopingcough_17.wav', 'whoopingcough_44.wav', 'unknown_812.wav', 'unknown_1320.wav'], '56': ['whoopingcough_60.wav', 'whoopingcough_31.wav', 'whoopingcough_57.wav', 'unknown_1431.wav', 'unknown_707.wav'], '57': ['whoopingcough_7.wav', 'whoopingcough_11.wav', 'whoopingcough_50.wav', 'unknown_548.wav', 'unknown_918.wav'], '58': ['whoopingcough_8.wav', 'whoopingcough_46.wav', 'whoopingcough_40.wav', 'unknown_468.wav', 'unknown_730.wav'], '59': ['whoopingcough_9.wav', 'whoopingcough_29.wav', 'whoopingcough_45.wav', 'unknown_1197.wav', 'unknown_694.wav']}]


In [14]:
def recall(cm_train):
    recall_dict = {}
    print('recall')
    idx = 0
    for i in cm_train:
        samples = sum(i)
        recall = i[idx]/samples
        print(str(idx) + ': '+ str(recall))
        recall_dict[str(idx)] = recall
        idx += 1
    return recall_dict
    
def precision(cm_train):
    arr_sum = sum(cm_train, 0)
    pre_dict = {}
    print('precision')
    for i in range(len(cm_train)):
        pre = cm_train[i][i]/arr_sum[i]
        print(str(i) + ': '+ str(pre))
        pre_dict[str(i)] = pre
    return pre_dict

def f1(pre, recall):
    score_dict = {}
    for key in pre.keys():    
        score = 2*(pre[key]*recall[key])/(pre[key]+recall[key])
        score_dict[key] = score
    print('score: ', score_dict)

In [15]:

def predict(dataset, key): 
    '''predict a few-shot task. Return: predicted label and original label'''
#         print(dataset[key])
    audio = dataset[key]
    query1 = audio[0]
    #         print(audio1, audio2, audio3, audio5, audio6)
    support1, labels1 = get_support(audio[1:])
    #         print(labels1)
    #         print(support1, labels1)
    pred, label = predictLabel(query1, support1, labels1)
    #         print(pred, label)
    #         val_predict_label.append(pred)
    #         val_real_label.append(label)
    return pred, label
# predict(way2shot2_unk)
# predict(way2shot2_wh)
        

In [16]:
def predict_cough(way2shot2_unk, way2shot2_wh):
    #way2shot2_unk
    val_predict_label = []
    val_real_label = []
    dataset = way2shot2_unk
    for key in dataset.keys():
        pred, label = predict(dataset, key)
        val_predict_label.append(pred)
        val_real_label.append(label)

    #way2shot2_wh
    dataset = way2shot2_wh
    for key in dataset.keys():
        pred, label = predict(dataset, key)
        val_predict_label.append(pred)
        val_real_label.append(label)
    return val_real_label, val_predict_label


# TCResNet

In [16]:
model_path = "D:\\HoangNgan\\DoAn\\paper\\exp\\few-shot-no-fed\\2022-05-18 16_52_05_tcresnet_set1\\best_model.pt"
model = torch.load(model_path)

In [17]:
#way2shot2_unk
val_predict_label = []
val_real_label = []
dataset = check_data_unk
c = 0
for key in dataset.keys():
    pred, label = predict(dataset, key)
    val_predict_label.append(pred)
    val_real_label.append(label)
print(c)
#way2shot2_wh
dataset = check_data_wh
for key in dataset.keys():
    pred, label = predict(dataset, key)
    val_predict_label.append(pred)
    val_real_label.append(label)
val_true, val_pred = val_real_label ,val_predict_label
cm_train = confusion_matrix(val_true, val_pred) 
re = recall(cm_train)
pre = precision(cm_train)
f1(pre, re)

NameError: name 'check_data_unk' is not defined

In [22]:
real_labels = []
pred_labels = []
for i in range(len(unk)):
    real, pred = predict_cough(unk[i], wh[i])
    real_labels.append(real)
    pred_labels.append(pred)

In [26]:
for i in range(len(real_labels)):
    print('----------------------')
    print('Dataset ' + str(i))
    val_true, val_pred = real_labels[i] ,pred_labels[i]
    cm_train = confusion_matrix(val_true, val_pred) 
    re = recall(scm_train)
    pre = precision(cm_train)
    f1(pre, re)
    

----------------------
Dataset 0
recall
0: 0.9666666666666667
1: 0.95
precision
0: 0.9508196721311475
1: 0.9661016949152542
score:  {'0': 0.9586776859504132, '1': 0.957983193277311}
----------------------
Dataset 1
recall
0: 0.9666666666666667
1: 0.9333333333333333
precision
0: 0.9354838709677419
1: 0.9655172413793104
score:  {'0': 0.9508196721311476, '1': 0.9491525423728815}
----------------------
Dataset 2
recall
0: 0.9833333333333333
1: 0.9333333333333333
precision
0: 0.9365079365079365
1: 0.9824561403508771
score:  {'0': 0.9593495934959351, '1': 0.9572649572649572}
----------------------
Dataset 3
recall
0: 0.95
1: 0.95
precision
0: 0.95
1: 0.95
score:  {'0': 0.9500000000000001, '1': 0.9500000000000001}
----------------------
Dataset 4
recall
0: 0.95
1: 0.9333333333333333
precision
0: 0.9344262295081968
1: 0.9491525423728814
score:  {'0': 0.9421487603305784, '1': 0.9411764705882353}


# TCResNet: distance

# IGNORE

In [29]:
model_path = "D:\\HoangNgan\\DoAn\\paper\\exp\\few-shot-no-fed\\2022-05-28 095550_tcresnet_1set\\best_model.pt"
model = torch.load(model_path)

In [30]:
real_labels = []
pred_labels = []
for i in range(len(unk)):
    real, pred = predict_cough(unk[i], wh[i])
    real_labels.append(real)
    pred_labels.append(pred)

In [32]:
for i in range(len(real_labels)):
    print('----------------------')
    print('Dataset ' + str(i))
    val_true, val_pred = real_labels[i] ,pred_labels[i]
    cm_train = confusion_matrix(val_true, val_pred) 
    re = recall(cm_train)
    pre = precision(cm_train)
    f1(pre, re)
    

----------------------
Dataset 0
recall
0: 1.0
1: 1.0
precision
0: 1.0
1: 1.0
score:  {'0': 1.0, '1': 1.0}
----------------------
Dataset 1
recall
0: 1.0
1: 1.0
precision
0: 1.0
1: 1.0
score:  {'0': 1.0, '1': 1.0}
----------------------
Dataset 2
recall
0: 1.0
1: 1.0
precision
0: 1.0
1: 1.0
score:  {'0': 1.0, '1': 1.0}
----------------------
Dataset 3
recall
0: 1.0
1: 1.0
precision
0: 1.0
1: 1.0
score:  {'0': 1.0, '1': 1.0}
----------------------
Dataset 4
recall
0: 1.0
1: 1.0
precision
0: 1.0
1: 1.0
score:  {'0': 1.0, '1': 1.0}


# TCResNetFed

In [35]:
model_path = "D:\\HoangNgan\\DoAn\\paper\\exp\\few-fed\\2022-05-09 05_51_23\\best_model.pt"
model = torch.load(model_path)

In [36]:
real_labels = []
pred_labels = []
for i in range(len(unk)):
    real, pred = predict_cough(unk[i], wh[i])
    real_labels.append(real)
    pred_labels.append(pred)

In [44]:
for i in range(len(real_labels)):
    print('----------------------')
    print('Dataset ' + str(i))
    val_true, val_pred = real_labels[i] ,pred_labels[i]
    cm_train = confusion_matrix(val_true, val_pred) 
    re = recall(cm_train)
    pre = precision(cm_train)
    f1(pre, re)
    

----------------------
Dataset 0
recall
0: 0.65
1: 0.5666666666666667
precision
0: 0.6
1: 0.6181818181818182
score:  {'0': 0.624, '1': 0.591304347826087}
----------------------
Dataset 1
recall
0: 0.6833333333333333
1: 0.4666666666666667
precision
0: 0.5616438356164384
1: 0.5957446808510638
score:  {'0': 0.6165413533834587, '1': 0.5233644859813084}
----------------------
Dataset 2
recall
0: 0.75
1: 0.55
precision
0: 0.625
1: 0.6875
score:  {'0': 0.6818181818181818, '1': 0.6111111111111112}
----------------------
Dataset 3
recall
0: 0.7333333333333333
1: 0.5666666666666667
precision
0: 0.6285714285714286
1: 0.68
score:  {'0': 0.6769230769230768, '1': 0.6181818181818183}
----------------------
Dataset 4
recall
0: 0.7833333333333333
1: 0.6333333333333333
precision
0: 0.6811594202898551
1: 0.7450980392156863
score:  {'0': 0.7286821705426356, '1': 0.6846846846846847}


# ATT

In [45]:
model_path = "D:\\HoangNgan\\DoAn\\paper\\exp\\few-shot-no-fed\\2022-05-18 17_30_33_att_set1\\best_model.pt"
model = torch.load(model_path)

In [46]:
real_labels = []
pred_labels = []
for i in range(len(unk)):
    real, pred = predict_cough(unk[i], wh[i])
    real_labels.append(real)
    pred_labels.append(pred)

C:\Users\Admins\anaconda3\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
D:\Jupyter\Cough Classification\few-shot-ho-master\protonets\models\encoder\ResNet18_Attention.py:214: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(out)


In [47]:
for i in range(len(real_labels)):
    print('----------------------')
    print('Dataset ' + str(i))
    val_true, val_pred = real_labels[i] ,pred_labels[i]
    cm_train = confusion_matrix(val_true, val_pred) 
    re = recall(cm_train)
    pre = precision(cm_train)
    f1(pre, re)
    

----------------------
Dataset 0
recall
0: 0.65
1: 0.5666666666666667
precision
0: 0.6
1: 0.6181818181818182
score:  {'0': 0.624, '1': 0.591304347826087}
----------------------
Dataset 1
recall
0: 0.6833333333333333
1: 0.4666666666666667
precision
0: 0.5616438356164384
1: 0.5957446808510638
score:  {'0': 0.6165413533834587, '1': 0.5233644859813084}
----------------------
Dataset 2
recall
0: 0.75
1: 0.55
precision
0: 0.625
1: 0.6875
score:  {'0': 0.6818181818181818, '1': 0.6111111111111112}
----------------------
Dataset 3
recall
0: 0.7333333333333333
1: 0.5666666666666667
precision
0: 0.6285714285714286
1: 0.68
score:  {'0': 0.6769230769230768, '1': 0.6181818181818183}
----------------------
Dataset 4
recall
0: 0.7833333333333333
1: 0.6333333333333333
precision
0: 0.6811594202898551
1: 0.7450980392156863
score:  {'0': 0.7286821705426356, '1': 0.6846846846846847}


# ATT FED

In [48]:

model_path = "D:\\HoangNgan\\DoAn\\2022-03-03 07_56_11\\best_model.pt"
model = torch.load(model_path)

In [49]:
real_labels = []
pred_labels = []
for i in range(len(unk)):
    real, pred = predict_cough(unk[i], wh[i])
    real_labels.append(real)
    pred_labels.append(pred)

C:\Users\Admins\anaconda3\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
D:\Jupyter\Cough Classification\few-shot-ho-master\protonets\models\encoder\ResNet18_Attention.py:214: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(out)


In [51]:
for i in range(len(real_labels)):
    print('----------------------')
    print('Dataset ' + str(i))
    val_true, val_pred = real_labels[i] ,pred_labels[i]
    cm_train = confusion_matrix(val_true, val_pred) 
    re = recall(cm_train)
    pre = precision(cm_train)
    f1(pre, re)
    

----------------------
Dataset 0
recall
0: 0.9333333333333333
1: 0.9166666666666666
precision
0: 0.9180327868852459
1: 0.9322033898305084
score:  {'0': 0.9256198347107439, '1': 0.9243697478991596}
----------------------
Dataset 1
recall
0: 0.9666666666666667
1: 0.8666666666666667
precision
0: 0.8787878787878788
1: 0.9629629629629629
score:  {'0': 0.9206349206349207, '1': 0.912280701754386}
----------------------
Dataset 2
recall
0: 0.9666666666666667
1: 0.9333333333333333
precision
0: 0.9354838709677419
1: 0.9655172413793104
score:  {'0': 0.9508196721311476, '1': 0.9491525423728815}
----------------------
Dataset 3
recall
0: 0.95
1: 0.9333333333333333
precision
0: 0.9344262295081968
1: 0.9491525423728814
score:  {'0': 0.9421487603305784, '1': 0.9411764705882353}
----------------------
Dataset 4
recall
0: 0.95
1: 0.9333333333333333
precision
0: 0.9344262295081968
1: 0.9491525423728814
score:  {'0': 0.9421487603305784, '1': 0.9411764705882353}


# Updated version - Few-fed

ATT - KERNEL

In [21]:

model_path = "D:\\HoangNgan\\DoAn\\paper\\exp\\few-fed\\2022-05-21 07_33_52_att_kernel\\best_model.pt"
model = torch.load(model_path)

In [22]:
real_labels = []
pred_labels = []
for i in range(len(unk)):
    real, pred = predict_cough(unk[i], wh[i])
    real_labels.append(real)
    pred_labels.append(pred)

C:\Users\Admins\anaconda3\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
D:\Jupyter\Cough Classification\few-shot-ho-master\protonets\models\encoder\ResNet18_Attention_kernel_v2.py:216: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(out)


In [23]:
for i in range(len(real_labels)):
    print('----------------------')
    print('Dataset ' + str(i))
    val_true, val_pred = real_labels[i] ,pred_labels[i]
    cm_train = confusion_matrix(val_true, val_pred) 
    re = recall(cm_train)
    pre = precision(cm_train)
    f1(pre, re)
    

----------------------
Dataset 0
recall
0: 0.55
1: 0.5333333333333333
precision
0: 0.5409836065573771
1: 0.5423728813559322
score:  {'0': 0.5454545454545455, '1': 0.5378151260504201}
----------------------
Dataset 1
recall
0: 0.48333333333333334
1: 0.5
precision
0: 0.4915254237288136
1: 0.4918032786885246
score:  {'0': 0.48739495798319327, '1': 0.49586776859504134}
----------------------
Dataset 2
recall
0: 0.5
1: 0.4166666666666667
precision
0: 0.46153846153846156
1: 0.45454545454545453
score:  {'0': 0.48000000000000004, '1': 0.43478260869565216}
----------------------
Dataset 3
recall
0: 0.5333333333333333
1: 0.5833333333333334
precision
0: 0.5614035087719298
1: 0.5555555555555556
score:  {'0': 0.5470085470085471, '1': 0.5691056910569107}
----------------------
Dataset 4
recall
0: 0.4166666666666667
1: 0.6
precision
0: 0.5102040816326531
1: 0.5070422535211268
score:  {'0': 0.45871559633027525, '1': 0.549618320610687}


ATT - DILATION

In [24]:

model_path = "D:\\HoangNgan\\DoAn\\paper\\exp\\few-fed\\2022-05-21 22_43_14_att_dilation\\best_model.pt"
model = torch.load(model_path)

In [25]:
real_labels = []
pred_labels = []
for i in range(len(unk)):
    real, pred = predict_cough(unk[i], wh[i])
    real_labels.append(real)
    pred_labels.append(pred)

C:\Users\Admins\anaconda3\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
D:\Jupyter\Cough Classification\few-shot-ho-master\protonets\models\encoder\ResNet18_Attention_dilation_v2.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(out)


In [26]:
for i in range(len(real_labels)):
    print('----------------------')
    print('Dataset ' + str(i))
    val_true, val_pred = real_labels[i] ,pred_labels[i]
    cm_train = confusion_matrix(val_true, val_pred) 
    re = recall(cm_train)
    pre = precision(cm_train)
    f1(pre, re)
    

----------------------
Dataset 0
recall
0: 0.9833333333333333
1: 0.85
precision
0: 0.8676470588235294
1: 0.9807692307692307
score:  {'0': 0.9218749999999999, '1': 0.9107142857142857}
----------------------
Dataset 1
recall
0: 0.9333333333333333
1: 0.8666666666666667
precision
0: 0.875
1: 0.9285714285714286
score:  {'0': 0.9032258064516129, '1': 0.896551724137931}
----------------------
Dataset 2
recall
0: 0.95
1: 0.8666666666666667
precision
0: 0.8769230769230769
1: 0.9454545454545454
score:  {'0': 0.912, '1': 0.9043478260869566}
----------------------
Dataset 3
recall
0: 0.9666666666666667
1: 0.8333333333333334
precision
0: 0.8529411764705882
1: 0.9615384615384616
score:  {'0': 0.90625, '1': 0.8928571428571429}
----------------------
Dataset 4
recall
0: 0.95
1: 0.85
precision
0: 0.8636363636363636
1: 0.9444444444444444
score:  {'0': 0.9047619047619048, '1': 0.8947368421052632}


ATT-KERNEL_DILATION-FED

In [27]:


model_path = "D:\\HoangNgan\\DoAn\\paper\\exp\\few-fed\\2022-05-23 07_11_45_att_kernel_dilation\\best_model.pt"
model = torch.load(model_path)

In [28]:
real_labels = []
pred_labels = []
for i in range(len(unk)):
    real, pred = predict_cough(unk[i], wh[i])
    real_labels.append(real)
    pred_labels.append(pred)

C:\Users\Admins\anaconda3\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
D:\Jupyter\Cough Classification\few-shot-ho-master\protonets\models\encoder\ResNet18_Attention_kernel_dilation_v2.py:218: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(out)


In [29]:
for i in range(len(real_labels)):
    print('----------------------')
    print('Dataset ' + str(i))
    val_true, val_pred = real_labels[i] ,pred_labels[i]
    cm_train = confusion_matrix(val_true, val_pred) 
    re = recall(cm_train)
    pre = precision(cm_train)
    f1(pre, re)
    

----------------------
Dataset 0
recall
0: 0.5
1: 0.6166666666666667
precision
0: 0.5660377358490566
1: 0.5522388059701493
score:  {'0': 0.5309734513274337, '1': 0.5826771653543308}
----------------------
Dataset 1
recall
0: 0.5333333333333333
1: 0.5
precision
0: 0.5161290322580645
1: 0.5172413793103449
score:  {'0': 0.5245901639344263, '1': 0.5084745762711865}
----------------------
Dataset 2
recall
0: 0.5
1: 0.65
precision
0: 0.5882352941176471
1: 0.5652173913043478
score:  {'0': 0.5405405405405405, '1': 0.6046511627906976}
----------------------
Dataset 3
recall
0: 0.5
1: 0.48333333333333334
precision
0: 0.4918032786885246
1: 0.4915254237288136
score:  {'0': 0.49586776859504134, '1': 0.48739495798319327}
----------------------
Dataset 4
recall
0: 0.5333333333333333
1: 0.55
precision
0: 0.5423728813559322
1: 0.5409836065573771
score:  {'0': 0.5378151260504201, '1': 0.5454545454545455}


# Attention - kernel (ver 2)

In [54]:


model_path = "D:\\HoangNgan\\DoAn\\paper\\exp\\few-fed\\2022-05-29 05_11_56_attention_kernel\\best_model.pt"
model = torch.load(model_path)

In [55]:
real_labels = []
pred_labels = []
for i in range(len(unk)):
    real, pred = predict_cough(unk[i], wh[i])
    real_labels.append(real)
    pred_labels.append(pred)

In [56]:
for i in range(len(real_labels)):
    print('----------------------')
    print('Dataset ' + str(i))
    val_true, val_pred = real_labels[i] ,pred_labels[i]
    cm_train = confusion_matrix(val_true, val_pred) 
    re = recall(cm_train)
    pre = precision(cm_train)
    f1(pre, re)
    

----------------------
Dataset 0
recall
0: 0.65
1: 0.5166666666666667
precision
0: 0.5735294117647058
1: 0.5961538461538461
score:  {'0': 0.609375, '1': 0.5535714285714286}
----------------------
Dataset 1
recall
0: 0.6666666666666666
1: 0.45
precision
0: 0.547945205479452
1: 0.574468085106383
score:  {'0': 0.6015037593984963, '1': 0.5046728971962617}
----------------------
Dataset 2
recall
0: 0.7
1: 0.5166666666666667
precision
0: 0.5915492957746479
1: 0.6326530612244898
score:  {'0': 0.6412213740458015, '1': 0.5688073394495414}
----------------------
Dataset 3
recall
0: 0.6333333333333333
1: 0.5333333333333333
precision
0: 0.5757575757575758
1: 0.5925925925925926
score:  {'0': 0.6031746031746033, '1': 0.5614035087719299}
----------------------
Dataset 4
recall
0: 0.6166666666666667
1: 0.5666666666666667
precision
0: 0.5873015873015873
1: 0.5964912280701754
score:  {'0': 0.6016260162601625, '1': 0.5811965811965812}


# 6 classes -> train; 2 classes -> test
# ATT FED

In [17]:


model_path = "D:\\HoangNgan\\DoAn\\paper\\exp\\few-fed\\2022-06-02 17_35_04_attention_6classes_2classes\\best_model.pt"
model = torch.load(model_path)

In [18]:
real_labels = []
pred_labels = []
for i in range(len(unk)):
    real, pred = predict_cough(unk[i], wh[i])
    real_labels.append(real)
    pred_labels.append(pred)

C:\Users\Admins\anaconda3\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
D:\Jupyter\Cough Classification\few-shot-ho-master\protonets\models\encoder\ResNet18_Attention.py:214: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(out)


In [19]:
for i in range(len(real_labels)):
    print('----------------------')
    print('Dataset ' + str(i))
    val_true, val_pred = real_labels[i] ,pred_labels[i]
    cm_train = confusion_matrix(val_true, val_pred) 
    re = recall(cm_train)
    pre = precision(cm_train)
    f1(pre, re)
    

----------------------
Dataset 0
recall
0: 0.8
1: 0.7
precision
0: 0.7272727272727273
1: 0.7777777777777778
score:  {'0': 0.761904761904762, '1': 0.7368421052631577}
----------------------
Dataset 1
recall
0: 0.85
1: 0.6666666666666666
precision
0: 0.7183098591549296
1: 0.8163265306122449
score:  {'0': 0.7786259541984734, '1': 0.7339449541284403}
----------------------
Dataset 2
recall
0: 0.8166666666666667
1: 0.7
precision
0: 0.7313432835820896
1: 0.7924528301886793
score:  {'0': 0.7716535433070867, '1': 0.743362831858407}
----------------------
Dataset 3
recall
0: 0.7166666666666667
1: 0.7
precision
0: 0.7049180327868853
1: 0.711864406779661
score:  {'0': 0.7107438016528925, '1': 0.7058823529411764}
----------------------
Dataset 4
recall
0: 0.75
1: 0.7166666666666667
precision
0: 0.7258064516129032
1: 0.7413793103448276
score:  {'0': 0.7377049180327869, '1': 0.728813559322034}
